# Library

In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from datetime import datetime
import math

In [2]:
# 소수점 옵션
pd.options.display.float_format = '{:.2f}'.format
# pd.reset_option('display.float_format')

# fold path

In [3]:
drive = 'D'

In [135]:
ods_fold = f'{drive}:/data/big2/ODS'
big2_fold = f'{drive}:/data/big2'
pre_fold = f'{drive}:/data/seoul_lab/pre'

# Load
- 등록&제원
- 저감장치 부착이력
- 법정동코드
- KOSIS

## 등록&제원

In [5]:
# 전체 컬럼명
'''
'차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '현재회원번호', '차대번호', '최초등록일자', '차량연식', '차량제작일자', '배출가스검사유효일자', '제원관리번호', 
'배출가스인증번호', '배출가스등급', '저공해조치YN(미사용)', '저공해조치종류', '저공해조치구조변경YN', '차량등록번호숫자정보', '차량말소YN', '최초회원번호', '취득일자', '취득금액', '생성일시_등록정보', '수정일시_등록정보', 
'차명', '차종_제원정보', '차종분류', '차종유형(소분류)', '자동차형식', '제작사명', '연료', '엔진형식', '공차중량', '총중량', '적재중량', '엔진출력', '배기량', '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', 
'삭제YN', '승인일자', '구동형식', '변속기종류', '생성일시_제원정보', '수정일시_제원정보'
'''

"\n'차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '현재회원번호', '차대번호', '최초등록일자', '차량연식', '차량제작일자', '배출가스검사유효일자', '제원관리번호', \n'배출가스인증번호', '배출가스등급', '저공해조치YN(미사용)', '저공해조치종류', '저공해조치구조변경YN', '차량등록번호숫자정보', '차량말소YN', '최초회원번호', '취득일자', '취득금액', '생성일시_등록정보', '수정일시_등록정보', \n'차명', '차종_제원정보', '차종분류', '차종유형(소분류)', '자동차형식', '제작사명', '연료', '엔진형식', '공차중량', '총중량', '적재중량', '엔진출력', '배기량', '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', \n'삭제YN', '승인일자', '구동형식', '변속기종류', '생성일시_제원정보', '수정일시_제원정보'\n"

In [6]:
# about 4m 30.7s
name = '[ODS]등록정보&제원정보(2022.12.28)'
file_name = f'{name}.csv'
smig = pd.read_csv(os.path.join(ods_fold, file_name), low_memory=False, encoding='cp949', usecols=['차대번호', '본거지법정동코드', '차종_등록정보', '차량연식', '배출가스인증번호', '배출가스등급', '차량말소YN', '차종유형(소분류)', '연료'])
smig.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29534047 entries, 0 to 29534046
Data columns (total 9 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   본거지법정동코드   int64 
 1   차종_등록정보    object
 2   차대번호       object
 3   차량연식       int64 
 4   배출가스인증번호   object
 5   배출가스등급     object
 6   차량말소YN     object
 7   차종유형(소분류)  object
 8   연료         object
dtypes: int64(2), object(7)
memory usage: 2.0+ GB


In [7]:
smig.columns

Index(['본거지법정동코드', '차종_등록정보', '차대번호', '차량연식', '배출가스인증번호', '배출가스등급', '차량말소YN',
       '차종유형(소분류)', '연료'],
      dtype='object')

In [8]:
smig['차종유형(소분류)'].unique()

array(['소형', '경형', '중형', '대형', nan], dtype=object)

In [9]:
smig1 = smig.rename(columns={'본거지법정동코드':'법정동코드', '차종_등록정보':'차종', '차종유형(소분류)':'차종유형'})
smig1.columns

Index(['법정동코드', '차종', '차대번호', '차량연식', '배출가스인증번호', '배출가스등급', '차량말소YN', '차종유형',
       '연료'],
      dtype='object')

In [10]:
smig1['차량말소YN'].value_counts(dropna=False)

차량말소YN
N    25693532
Y     3840515
Name: count, dtype: int64

In [11]:
smig2 = smig1[smig1['차량말소YN'] == 'N'].reset_index(drop=True)
smig2.shape

(25693532, 9)

## 저감장치 부착이력(중복차대제거)

In [12]:
# about 28.3s
# columns = 저감장치종류, TDMS부착여부, 구조변경일자, 부착업체명, 부착일자, 탈거승인일, 차량번호, 차대번호, 저감장치반납여부, 저감장치구분, 삼원촉매장치반납여부
name = '[ODS]저감장치부착이력(중복차대제거)(2022.12.28)'
file_name = f'{name}.csv'
att = pd.read_csv(os.path.join(ods_fold, file_name), low_memory=False, encoding='cp949', usecols=['차대번호', '저감장치구분'])
att.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 809255 entries, 0 to 809254
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   차대번호    809255 non-null  object
 1   저감장치구분  809255 non-null  object
dtypes: object(2)
memory usage: 12.3+ MB


In [215]:
att['저감장치구분'].unique()

array(['엔진교체', '3종', '1종', '2종', '엔진개조', '1종+SCR', '삼원촉매'], dtype=object)

In [216]:
att.loc[(att['저감장치구분'] == '1종') | (att['저감장치구분'] == '1종+SCR'), '저감장치부착유무'] = '유'

In [217]:
att['저감장치부착유무'].value_counts(dropna=False)

저감장치부착유무
NaN    512654
유      296601
Name: count, dtype: int64

## 법정동코드

In [13]:
# about 5.6s
# columns = '법정동코드', '시도', '시구군', '읍면동', '리', '지역3', '폐지여부', '대기관리권역', '대기관리권역YN' 
name = '법정동코드_2022.10.14(수정2023.05.04)'
file_name = f'{name}.xlsx'
code = pd.read_excel(os.path.join(big2_fold, file_name), usecols=['법정동코드', '시도', '시구군'])
code.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46342 entries, 0 to 46341
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   법정동코드   46342 non-null  int64 
 1   시도      46342 non-null  object
 2   시구군     46319 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.1+ MB


## KOSIS(2021)

In [14]:
# about 0.2s
name = '용도별_차종별_시군구별_자동차1일평균주행거리(2021)'
file_name = f'{name}.xlsx'
kos = pd.read_excel(os.path.join(ods_fold, file_name))
kos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1230 entries, 0 to 1229
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   시도명     1230 non-null   object 
 1   시군구명    1230 non-null   object 
 2   차종      1230 non-null   object 
 3   전체      1230 non-null   float64
dtypes: float64(1), object(3)
memory usage: 38.6+ KB


# 전처리

## 등급 수정

In [15]:
smig2['배출가스등급'].value_counts(dropna=False)

배출가스등급
2     10960244
3      8623130
1      2771395
4      1511214
5      1154733
X       288966
T2      247483
T3      136301
T4          37
T5          29
Name: count, dtype: int64

In [16]:
smig2['배출가스등급'].unique()

array(['5', 'X', 'T5', '4', '3', 'T4', 'T3', '2', '1', 'T2'], dtype=object)

In [17]:
smig2.loc[smig2['배출가스등급'] == '1', '등급'] = '1'
smig2.loc[(smig2['배출가스등급'] == '2') | (smig2['배출가스등급'] == 'T2'), '등급'] = '2'
smig2.loc[(smig2['배출가스등급'] == '3') | (smig2['배출가스등급'] == 'T3'), '등급'] = '3'
smig2.loc[(smig2['배출가스등급'] == '4') | (smig2['배출가스등급'] == 'T4'), '등급'] = '4'
smig2.loc[(smig2['배출가스등급'] == '5') | (smig2['배출가스등급'] == 'T5'), '등급'] = '5'
smig2.loc[smig2['배출가스등급'] == 'X', '등급'] = 'X'

In [18]:
smig2['등급'].value_counts(dropna=False)

등급
2    11207727
3     8759431
1     2771395
4     1511251
5     1154762
X      288966
Name: count, dtype: int64

## 연료 수정

In [19]:
smig2['연료'].value_counts(dropna=False)

연료
휘발유            12177033
경유              9890990
LPG(액화석유가스)     1904188
휘발유 하이브리드       1095741
전기               368693
NaN              146798
경유 하이브리드          35779
CNG(압축천연가스)       30461
수소                28227
LPG 하이브리드         14352
기타연료                933
CNG 하이브리드           304
LNG(액화천연가스)          27
알코올                   3
태양열                   2
등유                    1
Name: count, dtype: int64

In [20]:
smig2['연료'].unique()

array(['휘발유', '경유', nan, '기타연료', '등유', '알코올', 'LPG(액화석유가스)',
       'CNG(압축천연가스)', '휘발유 하이브리드', 'LPG 하이브리드', '전기', 'LNG(액화천연가스)',
       '경유 하이브리드', 'CNG 하이브리드', '수소', '태양열'], dtype=object)

In [22]:
# about 22.9s
smig2.loc[(smig2['연료'] == '휘발유') | (smig2['연료'] == '휘발유 하이브리드'), 'fuel'] = '휘발유'
smig2.loc[(smig2['연료'] == '경유') | (smig2['연료'] == '경유 하이브리드'), 'fuel'] = '경유'
smig2.loc[(smig2['연료'] == 'LPG(액화석유가스)') | (smig2['연료'] == 'LPG 하이브리드'), 'fuel'] = 'LPG'
smig2.loc[(smig2['연료'] == 'CNG(압축천연가스)') | (smig2['연료'] == 'CNG 하이브리드'), 'fuel'] = 'CNG'
smig2.loc[(smig2['연료'] == 'LNG(액화천연가스)') | (smig2['연료'] == 'LNG 하이브리드'), 'fuel'] = 'LNG'
smig2.loc[smig2['연료'] == '전기', 'fuel'] = '전기'
smig2.loc[smig2['연료'] == '수소', 'fuel'] = '수소'
smig2.loc[smig2['연료'] == '태양열', 'fuel'] = '태양열'
smig2.loc[smig2['연료'] == '알코올', 'fuel'] = '알코올'
smig2.loc[smig2['연료'] == '등유', 'fuel'] = '등유'
smig2.loc[smig2['연료'] == '기타연료', 'fuel'] = '기타연료'
smig2.loc[smig2['연료'].isnull() == True, 'fuel'] = np.nan

In [23]:
smig2['fuel'].value_counts(dropna=False)

fuel
휘발유     13272774
경유       9926769
LPG      1918540
전기        368693
NaN       146798
CNG        30765
수소         28227
기타연료         933
LNG           27
알코올            3
태양열            2
등유             1
Name: count, dtype: int64

In [28]:
smig2.shape

(25693532, 11)

In [29]:
smig2['fuel'].value_counts(dropna=False).sum()

25693532

## 중복 차대번호 제거

In [42]:
smig2.shape[0], len(smig2['차대번호'].unique())

(25693532, 25692512)

In [145]:
smig3 = smig2.drop_duplicates('차대번호').reset_index(drop=True)
smig3.shape

(25692512, 11)

## 차령 정보 계산 및 데이터 추가

In [146]:
current_yr = int(datetime.today().strftime("%Y"))
current_yr

2023

In [147]:
smig3.columns

Index(['법정동코드', '차종', '차대번호', '차량연식', '배출가스인증번호', '배출가스등급', '차량말소YN', '차종유형',
       '연료', '등급', 'fuel'],
      dtype='object')

In [148]:
smig3['차령'] = current_yr - smig3['차량연식']
smig3['차령'].describe()

count   25692512.00
mean           8.55
std            6.21
min            0.00
25%            4.00
50%            7.00
75%           12.00
max          123.00
Name: 차령, dtype: float64

In [149]:
smig3['차령'].isnull().sum()

0

In [150]:
smig3.columns

Index(['법정동코드', '차종', '차대번호', '차량연식', '배출가스인증번호', '배출가스등급', '차량말소YN', '차종유형',
       '연료', '등급', 'fuel', '차령'],
      dtype='object')

## 법정동코드로 지역정보 부착
- 시군구명 앞지역명만 추출

In [151]:
smig3.shape

(25692512, 12)

In [152]:
smig4 = smig3.merge(code, on='법정동코드', how='left')
smig4.shape

(25692512, 14)

In [153]:
# 25.9s
smig4.loc[smig4['시구군'].str.split(' ').str.len() != 1, '시구군'].head()

4     창원시 의창구
11    청주시 상당구
37    안산시 단원구
74     포항시 남구
75     포항시 남구
Name: 시구군, dtype: object

In [154]:
smig4.loc[smig4['시구군'].str.contains('부산') == True, '시구군'].head()

Series([], Name: 시구군, dtype: object)

In [155]:
# 58.3s
smig4['시구군'] = smig4['시구군'].str.split(' ').str[0]
smig4.loc[smig4['시구군'].str.split(' ').str.len() != 1, '시구군'].head()

6195     NaN
7506     NaN
7598     NaN
9987     NaN
21107    NaN
Name: 시구군, dtype: object

In [156]:
smig4['시구군'].head()

0    서초구
1     중구
2    남동구
3    원주시
4    창원시
Name: 시구군, dtype: object

## 1일평균주행거리 정보 추가
- 매칭 컬럼(등록&제원|kosis)
- 1\. 시도, 시구군, 차종 | 시도명, 시군구명, 차종
- 2\. 시도, 차종 | 시도명, 차종
- 3\. 시도, 시구군, 차종(nan) | 시도명, 시군구명, 차종(합계)
    - 차종:nan인 데이터는 kosis의 차종:합계의 일일평균주행거리로 매칭

In [157]:
smig4.columns

Index(['법정동코드', '차종', '차대번호', '차량연식', '배출가스인증번호', '배출가스등급', '차량말소YN', '차종유형',
       '연료', '등급', 'fuel', '차령', '시도', '시구군'],
      dtype='object')

In [158]:
kos.columns

Index(['시도명', '시군구명', '차종', '전체'], dtype='object')

In [159]:
smig4.shape

(25692512, 14)

In [160]:
# about 9.1s
# 시도, 시구군, 차종으로 매칭
smig5 = smig4.merge(kos, left_on=['시도', '시구군', '차종'], right_on=['시도명', '시군구명', '차종'], how='left')
smig5.shape

(25692512, 17)

In [161]:
smig5['전체'].isnull().sum()

341953

In [162]:
# 4.8s
smig5y = smig5.loc[smig5['전체'].isnull() == False]
smig5n = smig5.loc[smig5['전체'].isnull() == True]
smig5y.shape, smig5n.shape

((25350559, 17), (341953, 17))

In [163]:
smig5n.columns

Index(['법정동코드', '차종', '차대번호', '차량연식', '배출가스인증번호', '배출가스등급', '차량말소YN', '차종유형',
       '연료', '등급', 'fuel', '차령', '시도', '시구군', '시도명', '시군구명', '전체'],
      dtype='object')

In [164]:
smig5n = smig5n.drop(['시도명', '시군구명', '전체'], axis=1)
smig5n.columns

Index(['법정동코드', '차종', '차대번호', '차량연식', '배출가스인증번호', '배출가스등급', '차량말소YN', '차종유형',
       '연료', '등급', 'fuel', '차령', '시도', '시구군'],
      dtype='object')

In [165]:
smig5n.shape

(341953, 14)

In [166]:
kos.shape

(1230, 4)

In [167]:
kos.loc[kos['시군구명'] == '소계'].shape

(85, 4)

In [168]:
# 시도, 차종으로만 매칭
smig5nm = smig5n.merge(kos.loc[kos['시군구명'] == '소계'].reset_index(drop=True), left_on=['시도', '차종'], right_on=['시도명', '차종'], how='left')
smig5nm.shape

(341953, 17)

In [169]:
smig5nm.columns

Index(['법정동코드', '차종', '차대번호', '차량연식', '배출가스인증번호', '배출가스등급', '차량말소YN', '차종유형',
       '연료', '등급', 'fuel', '차령', '시도', '시구군', '시도명', '시군구명', '전체'],
      dtype='object')

In [170]:
smig5nm['전체'].isnull().sum()

14

In [171]:
smig5nm.loc[smig5nm['전체'].isnull() == True, ['시도', '시구군', '차종']]

,시도,시구군,차종
40,서울특별시,서초구,NaN
45,경상남도,함안군,NaN
201,경상남도,의령군,NaN
263,경상남도,함안군,NaN
372,전라남도,순천시,NaN
376,대구광역시,달성군,NaN
444,울산광역시,북구,NaN
512,경상남도,진주시,NaN
636,경상남도,창원시,NaN
650,경상북도,경주시,NaN


In [172]:
# 차종이 빈값인 데이터는 kosis의 차종이 합계인 데이터로 매칭
for a, b, c in smig5nm.loc[smig5nm['전체'].isnull() == True, ['시도', '시구군', '차종']].values:
    smig5nm.loc[(smig5nm['전체'].isnull() == True) & (smig5nm['시도'] == a) & ((smig5nm['시구군'] == b) & ((smig5nm['차종'].isnull() == True))), '전체'] = kos.loc[(kos['시도명'] == a) & (kos['시군구명'] == b) & (kos['차종'] == '합계'), '전체'].iloc[0]

In [173]:
kos.loc[(kos['시도명'] == a) & (kos['시군구명'] == b) & (kos['차종'] == '합계'), '전체'].iloc[0]

36.9

In [174]:
smig5nm['전체'].isnull().sum()

0

In [206]:
# 6.4s
smig5 = pd.concat([smig5y, smig5nm], ignore_index=True)
smig5.shape

(25692512, 17)

In [207]:
smig5.columns

Index(['법정동코드', '차종', '차대번호', '차량연식', '배출가스인증번호', '배출가스등급', '차량말소YN', '차종유형',
       '연료', '등급', 'fuel', '차령', '시도', '시구군', '시도명', '시군구명', '전체'],
      dtype='object')

In [208]:
# 12.1s
smig5 = smig5.rename(columns={'전체':'일일평균주행거리'})
smig5 = smig5.drop(['시도명', '시군구명'], axis=1)
smig5.columns

Index(['법정동코드', '차종', '차대번호', '차량연식', '배출가스인증번호', '배출가스등급', '차량말소YN', '차종유형',
       '연료', '등급', 'fuel', '차령', '시도', '시구군', '일일평균주행거리'],
      dtype='object')

In [209]:
smig5['일일평균주행거리'].isnull().sum()

0

## 저감장치 정보 추가

In [218]:
att.columns

Index(['차대번호', '저감장치구분', '저감장치부착유무'], dtype='object')

In [219]:
smig5.columns

Index(['법정동코드', '차종', '차대번호', '차량연식', '배출가스인증번호', '배출가스등급', '차량말소YN', '차종유형',
       '연료', '등급', 'fuel', '차령', '시도', '시구군', '일일평균주행거리'],
      dtype='object')

In [220]:
smig5.shape

(25692512, 15)

In [221]:
# 40.9s
smig6 = smig5.merge(att[['차대번호', '저감장치부착유무']], on='차대번호', how='left')
smig6.shape

(25692512, 16)

In [222]:
smig6['저감장치부착유무'].value_counts(dropna=False)

저감장치부착유무
NaN    25549321
유        143191
Name: count, dtype: int64

## [출력] 전처리 완료

In [223]:
# 2m 47.1s
smig6.to_csv(os.path.join(pre_fold, '전처리완료_일일평균주행거리포함.csv'), index=False)

# [로드] 전처리

In [ ]:
# # 
# name = '전처리완료_일일평균주행거리포함'
# file_name = f'{name}.csv'
# smig4 = pd.read_csv(os.path.join(pre_fold, file_name), low_memory=False)
# smig4.info()

## 연료로 분류

In [224]:
# 10.2s
# 휘발유, 경유, LPG
dgl = smig6.loc[(smig6['fuel'] == '휘발유') | (smig6['fuel'] == '경유') | (smig6['fuel'] == 'LPG')].reset_index(drop=True)
dgl.shape

(25117108, 16)

In [225]:
# 4.6s
# 나머지 : 전기, 수소, 태양열, CNG, LNG, 등유, 알코올, 기타연료, nan
etc = smig6[(smig6['fuel'] != '휘발유') & (smig6['fuel'] != '경유') & (smig6['fuel'] != 'LPG')].reset_index(drop=True)
etc.shape

(575404, 16)

In [226]:
dgl.shape[0] + etc.shape[0]

25692512

# 배출량 계산

## 자동차-엔진 가열(Hot-start) 배출

### 배출량 식
$$E_{i, j} = VKT \times {EF_i \over {1000}} \times DF \times (1 - {R \over {100}})$$

$E_{i, j}$ : 자동차 $j$의 도로주행시 발생한 오염물질 $i$의 배출량($kg/yr$)

$VKT$ : 주행거리($km/yr$)

$EF_i$ : 차종별, 연료별, 연식별, 차속별 배출계수($g/km$)

$DF$ : 열화계수

$R$ : 저감장치 부착 효율(%)

### 주행거리(VKT)
- [현재 설정] "용도별_차종별_시군구별_자동차주행거리" 자료(KOSIS)(2021)의 1일 평균주행거리 * 365 = 주행거리(km/yr)

In [227]:
# 2.6s
df2 = dgl.copy()

In [228]:
df2['VKT'] = df2['일일평균주행거리'] * 365

In [229]:
df2['VKT'].isnull().sum()

0

### DF(열화계수)
- 필요한 정보 : 연료, 차종, 차종유형, 연식, 차령
- 조건 수정(2023.04.20, 최)
    - 특수 소형 = 승용 소형
    - 승합 대형 = 화물 대형

In [230]:
df2['fuel'].unique()

array(['휘발유', '경유', 'LPG'], dtype=object)

In [231]:
DF_col = ['fuel', '차종', '차종유형', '차량연식', '차령']

In [232]:
df2[DF_col].head()

,fuel,차종,차종유형,차량연식,차령
0,휘발유,승용,소형,1976,47
1,휘발유,승용,경형,1968,55
2,휘발유,승용,소형,1976,47
3,경유,화물,중형,1971,52
4,휘발유,승용,소형,1986,37


In [233]:
df2[DF_col].values[0]

array(['휘발유', '승용', '소형', 1976, 47], dtype=object)

#### ❗ DF(열화계수) 코드
- 연료, 차종, 차종유형 설정에 해당되지 않는 차량의 열화계수 값 설정
    - 현재 설정값 : np.nan
    - 고려 설정값 : 1

In [234]:
# about 1m 14.5s
DF_CO_list = []
DF_HC_list = []
DF_PM_list = []
DF_NOx_list = []
# '연료', '차종', '차종유형', '연식', '차령'
for fuel, car_type, car_size, car_birth, car_age in tqdm(df2[DF_col].values):    
    if fuel == '경유':
        if (car_type == '승용') or ( (car_type == '특수') and (car_size == '소형') ): # 최 확인(2023.04.20) : "특수 소형" -> "승용 소형" 조건으로 계산
            warranty = 5
            if car_age <= warranty:
                DF_CO = 1
                DF_HC = 1
                DF_PM = 1
                DF_NOx = 1
            else:
                DF_CO = 1 + (car_age - warranty)*0.05
                DF_HC = 1 + (car_age - warranty)*0.05
                DF_PM = 1 + (car_age - warranty)*0.05
                DF_NOx = 1 + (car_age - warranty)*0.02
                if DF_CO > 1.5:
                    DF_CO = 1.5
                if DF_HC > 1.5:
                    DF_HC = 1.5
                if DF_PM > 1.5:
                    DF_PM = 1.5
                if DF_NOx > 1.2:
                    DF_NOx = 1.2
        elif car_type == '승합':
            if car_size == '소형' or car_size == '중형':
                warranty = 5
                if car_age <= warranty:
                    DF_CO = 1
                    DF_HC = 1
                    DF_PM = 1
                    DF_NOx = 1
                else:
                    DF_CO = 1 + (car_age - warranty)*0.05
                    DF_HC = 1 + (car_age - warranty)*0.05
                    DF_PM = 1 + (car_age - warranty)*0.05
                    DF_NOx = 1 + (car_age - warranty)*0.02
                    if DF_CO > 1.5:
                        DF_CO = 1.5
                    if DF_HC > 1.5:
                        DF_HC = 1.5
                    if DF_PM > 1.5:
                        DF_PM = 1.5
                    if DF_NOx > 1.2:
                        DF_NOx = 1.2
            elif car_size == '대형':
                warranty = 3
                if car_age <= warranty:
                    DF_CO = 1
                    DF_HC = 1
                    DF_PM = 1
                    DF_NOx = 1
                else:
                    DF_CO = 1 + (car_age - warranty)*0.05
                    DF_HC = 1 + (car_age - warranty)*0.05
                    DF_PM = 1 + (car_age - warranty)*0.05
                    DF_NOx = 1 + (car_age - warranty)*0.02
                    if DF_CO > 1.5:
                        DF_CO = 1.5
                    if DF_HC > 1.5:
                        DF_HC = 1.5
                    if DF_PM > 1.5:
                        DF_PM = 1.5
                    if DF_NOx > 1.2:
                        DF_NOx = 1.2
        elif car_type == '화물':
            # car_size : 경, 소, 중, 대
            warranty = 5
            if car_age <= warranty:
                DF_CO = 1
                DF_HC = 1
                DF_PM = 1
                DF_NOx = 1
            else:
                DF_CO = 1 + (car_age - warranty)*0.05
                DF_HC = 1 + (car_age - warranty)*0.05
                DF_PM = 1 + (car_age - warranty)*0.05
                DF_NOx = 1 + (car_age - warranty)*0.02
                if DF_CO > 1.5:
                    DF_CO = 1.5
                if DF_HC > 1.5:
                    DF_HC = 1.5
                if DF_PM > 1.5:
                    DF_PM = 1.5
                if DF_NOx > 1.2:
                    DF_NOx = 1.2
        elif car_type == '특수':
            # car_size : 중, 대
            warranty = 5
            if car_age <= warranty:
                DF_CO = 1
                DF_HC = 1
                DF_PM = 1
                DF_NOx = 1
            else:
                DF_CO = 1 + (car_age - warranty)*0.05
                DF_HC = 1 + (car_age - warranty)*0.05
                DF_PM = 1 + (car_age - warranty)*0.05
                DF_NOx = 1 + (car_age - warranty)*0.02
                if DF_CO > 1.5:
                    DF_CO = 1.5
                if DF_HC > 1.5:
                    DF_HC = 1.5
                if DF_PM > 1.5:
                    DF_PM = 1.5
                if DF_NOx > 1.2:
                    DF_NOx = 1.2
    elif (fuel == '휘발유') or (fuel == 'LPG(액화석유가스)'):
        if car_type == '승용':
            # 경, 소, 중, 대
            if car_birth <= 2000:
                warranty = 5
                if car_age <= warranty:
                    DF_CO = 1
                    DF_HC = 1
                    DF_PM = 1
                    DF_NOx = 1
                else:
                    DF_CO = 1 + (car_age - warranty)*0.1
                    DF_HC = 1 + (car_age - warranty)*0.1
                    DF_PM = 1 + (car_age - warranty)*0.1
                    DF_NOx = 1 + (car_age - warranty)*0.1
                    if DF_CO > 2.0:
                        DF_CO = 2.0
                    if DF_HC > 2.0:
                        DF_HC = 2.0
                    if DF_PM > 2.0:
                        DF_PM = 2.0
                    if DF_NOx > 2.0:
                        DF_NOx = 2.0
            elif car_birth >= 2001:
                warranty = 10
                if car_age <= warranty:
                    DF_CO = 1
                    DF_HC = 1
                    DF_PM = 1
                    DF_NOx = 1
                else:
                    DF_CO = 1 + (car_age - warranty)*0.1
                    DF_HC = 1 + (car_age - warranty)*0.1
                    DF_PM = 1 + (car_age - warranty)*0.1
                    DF_NOx = 1 + (car_age - warranty)*0.1
                    if DF_CO > 2.0:
                        DF_CO = 2.0
                    if DF_HC > 2.0:
                        DF_HC = 2.0
                    if DF_PM > 2.0:
                        DF_PM = 2.0
                    if DF_NOx > 2.0:
                        DF_NOx = 2.0
        elif car_type == '승합':
            if car_size == '경형':
                warranty = 5
                if car_age <= warranty:
                    DF_CO = 1
                    DF_HC = 1
                    DF_PM = 1
                    DF_NOx = 1
                else:
                    DF_CO = 1 + (car_age - warranty)*0.1
                    DF_HC = 1 + (car_age - warranty)*0.1
                    DF_PM = 1 + (car_age - warranty)*0.1
                    DF_NOx = 1 + (car_age - warranty)*0.1
                    if DF_CO > 2.0:
                        DF_CO = 2.0
                    if DF_HC > 2.0:
                        DF_HC = 2.0
                    if DF_PM > 2.0:
                        DF_PM = 2.0
                    if DF_NOx > 2.0:
                        DF_NOx = 2.0
            elif (car_size == '소형') or (car_size == '중형'):
                if car_birth <= 2005:
                    warranty = 5
                    if car_age <= warranty:
                        DF_CO = 1
                        DF_HC = 1
                        DF_PM = 1
                        DF_NOx = 1
                    else:
                        DF_CO = 1 + (car_age - warranty)*0.1
                        DF_HC = 1 + (car_age - warranty)*0.1
                        DF_PM = 1 + (car_age - warranty)*0.1
                        DF_NOx = 1 + (car_age - warranty)*0.1
                        if DF_CO > 2.0:
                            DF_CO = 2.0
                        if DF_HC > 2.0:
                            DF_HC = 2.0
                        if DF_PM > 2.0:
                            DF_PM = 2.0
                        if DF_NOx > 2.0:
                            DF_NOx = 2.0
                elif car_birth >= 2006:
                    warranty = 10
                    if car_age <= warranty:
                        DF_CO = 1
                        DF_HC = 1
                        DF_PM = 1
                        DF_NOx = 1
                    else:
                        DF_CO = 1 + (car_age - warranty)*0.1
                        DF_HC = 1 + (car_age - warranty)*0.1
                        DF_PM = 1 + (car_age - warranty)*0.1
                        DF_NOx = 1 + (car_age - warranty)*0.1
                        if DF_CO > 2.0:
                            DF_CO = 2.0
                        if DF_HC > 2.0:
                            DF_HC = 2.0
                        if DF_PM > 2.0:
                            DF_PM = 2.0
                        if DF_NOx > 2.0:
                            DF_NOx = 2.0
        elif (car_type == '화물') or ( (car_type == '승합') and (car_size == '대형')): # 최 확인(2023.04.20) : 습합 대형 -> 화물 대형 조건으로 계산
            # 경, 소, 중, 대
            warranty = 5
            if car_age <= warranty:
                DF_CO = 1
                DF_HC = 1
                DF_PM = 1
                DF_NOx = 1
            else:
                DF_CO = 1 + (car_age - warranty)*0.1
                DF_HC = 1 + (car_age - warranty)*0.1
                DF_PM = 1 + (car_age - warranty)*0.1
                DF_NOx = 1 + (car_age - warranty)*0.1
                if DF_CO > 2.0:
                    DF_CO = 2.0
                if DF_HC > 2.0:
                    DF_HC = 2.0
                if DF_PM > 2.0:
                    DF_PM = 2.0
                if DF_NOx > 2.0:
                    DF_NOx = 2.0
        elif car_type == '특수':
            # 중, 대
            warranty = 5
            if car_age <= warranty:
                DF_CO = 1
                DF_HC = 1
                DF_PM = 1
                DF_NOx = 1
            else:
                DF_CO = 1 + (car_age - warranty)*0.1
                DF_HC = 1 + (car_age - warranty)*0.1
                DF_PM = 1 + (car_age - warranty)*0.1
                DF_NOx = 1 + (car_age - warranty)*0.1
                if DF_CO > 2.0:
                    DF_CO = 2.0
                if DF_HC > 2.0:
                    DF_HC = 2.0
                if DF_PM > 2.0:
                    DF_PM = 2.0
                if DF_NOx > 2.0:
                    DF_NOx = 2.0
    else:
        # 설정 고민
            # 고려 : 1
        # DF_CO, DF_HC, DF_PM, DF_NOx = np.nan, np.nan, np.nan, np.nan
        DF_CO, DF_HC, DF_PM, DF_NOx = 1, 1, 1, 1

    # 열화계수(DF) 리스트에 저장
    DF_CO_list.append(DF_CO)
    DF_HC_list.append(DF_HC)
    DF_PM_list.append(DF_PM)
    DF_NOx_list.append(DF_NOx)

100%|██████████| 25117108/25117108 [01:10<00:00, 355901.37it/s]


In [235]:
df2.shape[0]

25117108

In [236]:
len(DF_CO_list), len(DF_HC_list), len(DF_PM_list), len(DF_NOx_list)

(25117108, 25117108, 25117108, 25117108)

In [237]:
df2['DF_CO'] = DF_CO_list
df2['DF_HC'] = DF_HC_list
df2['DF_PM'] = DF_PM_list
df2['DF_NOx'] = DF_NOx_list

In [238]:
check_DF_col = ['DF_CO', 'DF_HC', 'DF_PM', 'DF_NOx']
df2.loc[df2['차령'] < 10, DF_col + check_DF_col].head()

,fuel,차종,차종유형,차량연식,차령,DF_CO,DF_HC,DF_PM,DF_NOx
6596372,경유,화물,대형,2014,9,1.20,1.20,1.20,1.08
7687743,휘발유,승용,대형,2014,9,1.00,1.00,1.00,1.00
8569447,경유,화물,대형,2014,9,1.20,1.20,1.20,1.08
8732660,경유,승용,중형,2015,8,1.15,1.15,1.15,1.06
8779297,휘발유,승용,중형,2014,9,1.00,1.00,1.00,1.00


### 저감장치 부착 효율(R) 계산
- 저감장치부착유무 : 유 -> DPF로 가정하고 진행
- 휘발성 유기 화합물(Volatile Organic Compounds:VOC) : 생활주변에서 흔히 사용하는 탄화수소류가 거의 해당됨.
- 저감장치별 물질 제거 효율(%)

    장치종류|CO|VOC|PM
    :-:|:-:|:-:|:-:
    DPF|99.5|90|83.6
    pDPF|94.6|89.3|56
    DOC|85.4|72|35

In [239]:
df2.loc[df2['저감장치부착유무'] == '유', ['R_CO', 'R_HC', 'R_PM']] = 99.5, 90, 83.6

In [240]:
df2.shape[0]

25117108

In [241]:
check_R_col = ['R_CO', 'R_HC', 'R_PM']

In [242]:
df2[check_R_col].notnull().sum()

R_CO    143185
R_HC    143185
R_PM    143185
dtype: int64

In [243]:
df2.loc[df2['저감장치부착유무'] == '유', DF_col + check_DF_col + check_R_col].head()

,fuel,차종,차종유형,차량연식,차령,DF_CO,DF_HC,DF_PM,DF_NOx,R_CO,R_HC,R_PM
40425,경유,특수,대형,1986,37,1.50,1.50,1.50,1.20,99.50,90.00,83.60
48341,경유,화물,대형,1986,37,1.50,1.50,1.50,1.20,99.50,90.00,83.60
53808,경유,화물,대형,1988,35,1.50,1.50,1.50,1.20,99.50,90.00,83.60
61857,경유,특수,대형,1988,35,1.50,1.50,1.50,1.20,99.50,90.00,83.60
63642,경유,화물,대형,1988,35,1.50,1.50,1.50,1.20,99.50,90.00,83.60


In [244]:
df2[check_R_col] = df2[check_R_col].fillna(0)

In [245]:
df2[check_R_col].isnull().sum()

R_CO    0
R_HC    0
R_PM    0
dtype: int64

### ❗ 배출계수(EFi)
- 연료, 차종, 차종유형 설정에 해당되지 않는 차량의 배출계수 값 설정
    - 현재 설정값 : np.nan
    - 고려 설정값 : 1
- 조건 수정(2023.04.20, 최)
    - 특수 중형 = 승합 중형
    - 화물 중형 = 승합 중형
    - 화물 대형 = 승합 대형
    - 특수 중형 = 승용 중형

In [246]:
EFi_col = ['차종', '차종유형', '연료', '차량연식']
# 참고 : KOSIS 차량속도(2017) 일반국도 평균 (https://kosis.kr/statHtml/statHtml.do?orgId=210&tblId=DT_21002_J008)
V = 54.1
# 국가 대기오염물질 배출량 산정방법 편람(V)(2022) 부록 참고(for PM-2.5)
k = 0.92

In [247]:
df2[EFi_col].values[0]

array(['승용', '소형', '휘발유', 1976], dtype=object)

In [248]:
df2['차종유형'].unique()

array(['소형', '경형', '중형', '대형', nan], dtype=object)

In [249]:
# about 7.9s
EFi_CO_list = []
EFi_HC_list = []
EFi_NOx_list = []
EFi_PM10_list = []
EFi_PM2_5_list = []

for car_type, car_size, fuel, car_birth in tqdm(df2[EFi_col].values):
    if fuel == '휘발유':
        if (car_type == '승용') and (car_size == '경형'):
            if car_birth <= 1996:
                if V <= 65:
                    EFi_CO = 59.783 * (V**-1.0007)
                else:
                    EFi_CO = 0.0874 * V - 3.5618
                EFi_HC = 7.6244 * (V**-0.8364)
                EFi_NOx = 2.6754 * (V**-0.3236)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 1997 <= car_birth <= 1999:
                if V <= 65:
                    EFi_CO = 59.783 * (V**-1.0007)
                else:
                    EFi_CO = 0.0874 * V - 3.5618
                EFi_HC = 8.6275 * (V**-1.0722)
                EFi_NOx = 3.2294 * (V**-0.5763)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2000 <= car_birth <= 2002:
                if V <= 65:
                    EFi_CO = 60.556 * (V**-1.2501)
                else:
                    EFi_CO = -0.0006 * V + 0.5753
                EFi_HC = 5.1835 * (V**-1.1889)
                EFi_NOx = 1.7525 * (V**-0.6481)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2003 <= car_birth <= 2005:
                if V <= 65:
                    EFi_CO = 60.556 * (V**-1.2501)
                else:
                    EFi_CO = -0.0006 * V + 0.5753
                EFi_HC = 0.7446 * (V**-0.9392)
                EFi_NOx = 0.3403 * (V**-0.5455)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2006 <= car_birth <= 2008:
                if V <= 45:
                    EFi_CO = 4.9952 * (V**-0.8461)
                else:
                    EFi_CO = -0.0001 * V**2 + 0.0229 * V - 0.5701
                EFi_HC = 0.2958 * (V**-0.7830)
                EFi_NOx = 0.4819 * (V**-0.9198)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2009 <= car_birth <= 2011:
                if V <= 45:
                    EFi_CO = 4.5956 * (V**-0.8461)
                else:
                    EFi_CO = -9.2000*(10**-5) * (V**2) + 2.1068*(10**-2) * V - 5.2449*(10**-1)
                EFi_HC = 0.2662 * (V**-0.7830)
                EFi_NOx = 0.4476 * (V**-0.9198)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2012 <= car_birth <= 2013:
                if V <= 45:
                    EFi_CO = 4.4517 * (V**-0.8461)
                else:
                    EFi_CO = -8.9120*(10**-5) * (V**2) + 2.0408*(10**-2)*V - 5.0807*(10**-1)
                EFi_HC = 0.2556 * (V**-0.7830)
                EFi_NOx = 0.4353 * (V**-0.9198)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif car_birth == 2014:
                if V <= 45:
                    EFi_CO = 4.3079 * (V**-0.8461)
                else:
                    EFi_CO = -8.6240*(10**-5) * (V**2) + 1.9749*(10**-2)*V - 4.9165*(10**-1)
                EFi_HC = 0.2449 * (V**-0.7830)
                EFi_NOx = 0.4230 * (V**-0.9198)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2015 <= car_birth:
                if V <= 45:
                    EFi_CO = 4.164 * (V**-0.8461)
                else:
                    EFi_CO = -8.3360*(10**-5) * (V**2) + 1.9089*(10**-2)*V - 4.7524*(10**-1)
                EFi_HC = 0.2343 * (V**-0.7830)
                EFi_NOx = 0.4106 * (V**-0.9198)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
        elif (car_type == '승용') and (car_size == '소형'):
            if car_birth <= 1986:
                EFi_CO = 247.00 * (V**-0.6651)
                EFi_HC = 15.953 * (V**-0.5059)
                EFi_NOx = 3.1140 * (V**-0.2278)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 1987 <= car_birth <= 1990:
                EFi_CO = 36.169 * (V**-0.7587)
                EFi_HC = 15.607 * (V**-1.0423)
                EFi_NOx = 6.2007 * (V**-0.6781)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 1991 <= car_birth <= 1999:
                EFi_CO = 111.67 * (V**-1.1566)
                EFi_HC = 32.017 * (V**-1.4171)
                EFi_NOx = 7.5244 * (V**-0.7634)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2000 <= car_birth <= 2002:
                EFi_CO = 22.356 * (V**-0.9068)
                EFi_HC = 0.8428 * (V**-0.8829)
                EFi_NOx = 1.2613 * (V**-0.3873)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2003 <= car_birth <= 2005:
                EFi_CO = 1.4898 * (V**-0.3837)
                EFi_HC = 0.1738 * (V**-0.7268)
                EFi_NOx = 0.1563 * (V**-0.2671)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2006 <= car_birth <= 2008:
                EFi_CO = 1.0000*(10**-4)*(V**2) - 7.1000*(10**-3)*V + 2.2450*(10**-1)
                if V <= 65.4:
                    EFi_HC = 0.0633 * (V**-1.0484)
                else:
                    EFi_HC = 1.3200*(10**-6)*(V**2) - 1.8800*(10**-4)*V + 7.7000*(10**-3)
                EFi_NOx = -3.5000*(10**-6)*(V**2) + 3.3000*(10**-4)*V + 1.1200*(10**-2)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2009 <= car_birth <= 2011:
                EFi_CO = 9.2000*(10**-5)*(V**2) - 6.5320*(10**-3)*V + 2.0654*(10**-1)
                if V <= 65.4:
                    EFi_HC = 0.0570 * (V**-1.0484)
                else:
                    EFi_HC = 1.1880*(10**-6)*(V**2) - 1.6920*(10**-4)*V + 6.9300*(10**-3)
                EFi_NOx = -3.2511*(10**-6)*(V**2) + 3.0653*(10**-4)*V + 1.0404*(10**-2)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2012 <= car_birth <= 2013:
                EFi_CO = 8.9120*(10**-5)*(V**2) - 6.3275*(10**-3)*V + 2.0007*(10**-1)
                if V <= 65.4:
                    EFi_HC = 0.0547 * (V**-1.0484)
                else:
                    EFi_HC = 1.1405*(10**-6)*(V**2) - 1.6243*(10**-4)*V + 6.6528*(10**-3)
                EFi_NOx = -3.1615*(10**-6)*(V**2) + 2.9809*(10**-4)*V + 1.0117*(10**-2)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif car_birth == 2014:
                EFi_CO = 8.6240*(10**-5)*(V**2) - 6.1230*(10**-3)*V + 1.9361*(10**-1)
                if V <= 65.4:
                    EFi_HC = 0.0524 * (V**-1.0484)
                else:
                    EFi_HC = 1.0930*(10**-6)*(V**2) - 1.5566*(10**-4)*V + 6.3756*(10**-3)
                EFi_NOx = -3.0719*(10**-6)*(V**2) + 2.8964*(10**-4)*V + 9.8301*(10**-3)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2015 <= car_birth:
                EFi_CO = 8.3360*(10**-5)*(V**2) - 5.9186*(10**-3)*V + 1.8714*(10**-1)
                if V <= 65.4:
                    EFi_HC = 0.0501 * (V**-1.0484)
                else:
                    EFi_HC = 1.0500*(10**-6)*(V**2) - 1.4890*(10**-4)*V + 6.09840*(10**-3)
                EFi_NOx = -2.9823*(10**-6)*(V**2) + 2.8119*(10**-4)*V + 9.5434*(10**-3)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
        elif ( (car_type == '승용') and (car_size == '중형') ) or ( (car_type == '특수') and (car_size == '중형') ): # 최 확인(2023.04.20) : 특수 중형 -> 승용 중형 조건으로 계산
            if car_birth <= 1986:
                EFi_CO = 247.00 * (V**-0.6651)
                EFi_HC = 15.953 * (V**-0.5059)
                EFi_NOx = 3.1140 * (V**-0.2278)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 1987 <= car_birth <= 1990:
                EFi_CO = 36.169 * (V**-0.7587)
                EFi_HC = 15.607 * (V**-1.0423)
                EFi_NOx = 6.2007 * (V**-0.6781)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 1991 <= car_birth <= 1999:
                EFi_CO = 51.555 * (V**-0.9531)
                EFi_HC = 31.816 * (V**-1.4804)
                EFi_NOx = 7.5244 * (V**-0.7634)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2000 <= car_birth <= 2002:
                EFi_CO = 29.921 * (V**-0.8868)
                EFi_HC = 7.9374 * (V**-1.3041)
                EFi_NOx = 1.8525 * (V**-0.4192)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2003 <= car_birth <= 2005:
                EFi_CO = 2.4938 * (V**-0.6106)
                EFi_HC = 0.4262 * (V**-1.0122)
                EFi_NOx = 0.1818 * (V**-0.4316)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2006 <= car_birth <= 2008:
                EFi_CO = 2.2900*(10**-5)*(V**2) - 1.6300*(10**-3)*V + 5.8300*(10**-2)
                if V <= 65.4:
                    EFi_HC = 0.0633 * (V**-1.0484)
                else:
                    EFi_HC = 1.3200*(10**-6)*(V**2) - 1.8800*(10**-4)*V + 7.7000*(10**-3)
                EFi_NOx = -3.5000*(10**-6)*(V**2) + 3.3000*(10**-4)*V + 1.1200*(10**-2)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2009 <= car_birth <= 2011:
                EFi_CO = 2.1068*(10**-5)*(V**2) - 1.4996*(10**-3)*V + 5.3636*(10**-2)
                if V <= 65.4:
                    EFi_HC = 0.0570 * (V**-1.0484)
                else:
                    EFi_HC = 1.1880*(10**-6)*(V**2) - 1.6920*(10**-4)*V + 6.9300*(10**-3)
                EFi_NOx = -3.2511*(10**-6)*(V**2) + 3.0653*(10**-4)*V + 1.0404*(10**-2)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2012 <= car_birth <= 2013:
                EFi_CO = 2.0408*(10**-5)*(V**2) - 1.4527*(10**-3)*V + 5.1957*(10**-2)
                if V <= 65.4:
                    EFi_HC = 0.0547 * (V**-1.0484)
                else:
                    EFi_HC = 1.1405*(10**-6)*(V**2) - 1.6243*(10**-4)*V + 6.6528*(10**-3)
                EFi_NOx = -3.1615*(10**-6)*(V**2) + 2.9809*(10**-4)*V + 1.0117*(10**-2)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif car_birth == 2014:
                EFi_CO = 1.9749*(10**-5)*(V**2) - 1.4057*(10**-3)*V + 5.0278*(10**-2)
                if V <= 65.4:
                    EFi_HC = 0.0524 * (V**-1.0484)
                else:
                    EFi_HC = 1.0930*(10**-6)*(V**2) - 1.5566*(10**-4)*V + 6.3756*(10**-3)
                EFi_NOx = -3.0719*(10**-6)*(V**2) + 2.8964*(10**-4)*V + 9.8301*(10**-3)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2015 <= car_birth:
                EFi_CO = 1.9089*(10**-5)*(V**2) - 1.3588*(10**-3)*V + 4.8599*(10**-2)
                if V <= 65.4:
                    EFi_HC = 0.0501 * (V**-1.0484)
                else:
                    EFi_HC = 1.0500*(10**-6)*(V**2) - 1.4890*(10**-4)*V + 6.0984*(10**-3)
                EFi_NOx = -2.9823*(10**-6)*(V**2) + 2.8119*(10**-4)*V + 9.5434*(10**-3)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
        elif (car_type == '승용') and (car_size == '대형'):
            if car_birth <= 1986:
                EFi_CO = 247.00 * (V**-0.6651)
                EFi_HC = 15.953 * (V**-0.5059)
                EFi_NOx = 3.1140 * (V**-0.2278)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 1987 <= car_birth <= 1990:
                EFi_CO = 36.169 * (V**-0.7587)
                EFi_HC = 15.607 * (V**-1.0423)
                EFi_NOx = 6.2007 * (V**-0.6781)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 1991 <= car_birth <= 1999:
                EFi_CO = 51.555 * (V**-0.9531)
                EFi_HC = 31.816 * (V**-1.4804)
                EFi_NOx = 7.5244 * (V**-0.7634)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2000 <= car_birth <= 2002:
                EFi_CO = 29.921 * (V**-0.8868)
                EFi_HC = 7.9374 * (V**-1.3041)
                EFi_NOx = 1.8525 * (V**-0.4192)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2003 <= car_birth <= 2005:
                EFi_CO = 2.4938 * (V**-0.6106)
                EFi_HC = 0.4262 * (V**-1.0122)
                EFi_NOx = 0.1818 * (V**-0.4316)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2006 <= car_birth <= 2008:
                if V <= 65.4:
                    EFi_CO = 1.4082 * (V**-0.7728)
                else:
                    EFi_CO = 8.0000*(10**-5)*(V**2) - 1.2700*(10**-2)*V + 5.7510*(10**-1)
                if V <= 65.4:
                    EFi_HC = 0.0633 * (V**-1.0484)
                else:
                    EFi_HC = 1.3200*(10**-6)*(V**2) - 1.8800*(10**-4)*V + 7.7000*(10**-3)
                EFi_NOx = -3.5000*(10**-6)*(V**2) + 3.3000*(10**-4)*V + 1.1200*(10**-2)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2009 <= car_birth <= 2011:
                if V <= 65.4:
                    EFi_CO = 1.2955 * (V**-0.7728)
                else:
                    EFi_CO = 7.3600*(10**-5)*(V**2) - 1.1684*(10**-2)*V + 5.2909*(10**-1)
                if V <= 65.4:
                    EFi_HC = 0.0570 * (V**-1.0484)
                else:
                    EFi_HC = 1.1880*(10**-6)*(V**2) - 1.6920*(10**-4)*V + 6.9300*(10**-3)
                EFi_NOx = -3.2511*(10**-6)*(V**2) + 3.0653*(10**-4)*V + 1.0404*(10**-2)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2012 <= car_birth <= 2013:
                if V <= 65.4:
                    EFi_CO = 1.2550 * (V**-0.7728)
                else:
                    EFi_CO = 7.1296*(10**-5)*(V**2) - 1.1318*(10**-2)*V + 5.1253*(10**-1)
                if V <= 65.4:
                    EFi_HC = 0.0547 * (V**-1.0484)
                else:
                    EFi_HC = 1.1405*(10**-6)*(V**2) - 1.6243*(10**-4)*V + 6.6528*(10**-3)
                EFi_NOx = -3.1615*(10**-6)*(V**2) + 2.9809*(10**-4)*V + 1.0117*(10**-2)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif car_birth == 2014:
                if V <= 65.4:
                    EFi_CO = 1.2144 * (V**-0.7728)
                else:
                    EFi_CO = 6.8992*(10**-5)*(V**2) - 1.0952*(10**-2)*V + 4.9597*(10**-1)
                if V <= 65.4:
                    EFi_HC = 0.0524 * (V**-1.0484)
                else:
                    EFi_HC = 1.0930*(10**-6)*(V**2) - 1.5566*(10**-4)*V + 6.3756*(10**-3)
                EFi_NOx = -3.0719*(10**-6)*(V**2) + 2.8964*(10**-4)*V + 9.8301*(10**-3)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2015 <= car_birth:
                if V <= 65.4:
                    EFi_CO = 1.1739 * (V**-0.7728)
                else:
                    EFi_CO = 6.6688*(10**-5)*(V**2) - 1.0587*(10**-2)*V + 4.7940*(10**-1)
                if V <= 65.4:
                    EFi_HC = 0.0501 * (V**-1.0484)
                else:
                    EFi_HC = 1.0500*(10**-6)*(V**2) - 1.4890*(10**-4)*V + 6.0984*(10**-3)
                EFi_NOx = -2.9823*(10**-6)*(V**2) + 2.8119*(10**-4)*V + 9.5434*(10**-3)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
        elif ( (car_type == '승합') and (car_size == '경형') ) or ( (car_type == '화물') and (car_size == '경형') ):
            if car_birth <= 1996:
                if V <= 45:
                    EFi_CO = 11.249 * (V**-0.6579)
                else:
                    EFi_CO = 0.0003 * (V**2) + 0.0002 * V + 0.4136
                EFi_HC = 7.6244 * (V**-0.8364)
                EFi_NOx = 2.6754 * (V**-0.3236)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 1997 <= car_birth <= 1999:
                if V <= 45:
                    EFi_CO = 16.965 * (V**-0.8461)
                else:
                    EFi_CO = -0.0003 * (V**2) + 0.0777 * V - 1.9363
                EFi_HC = 3.0285 * (V**-0.7830)
                EFi_NOx = 1.9923 * (V**-0.3889)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2000 <= car_birth <= 2002:
                if V <= 45:
                    EFi_CO = 9.9433 * (V**-0.8461)
                else:
                    EFi_CO = -0.0002 * (V**2) + 0.0455 * V - 1.1349
                EFi_HC = 1.8928 * (V**-0.7830)
                EFi_NOx = 1.2352 * (V**-0.3889)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2003 <= car_birth <= 2005:
                if V <= 45:
                    EFi_CO = 9.9433 * (V**-0.8461)
                else:
                    EFi_CO = -0.0002 * (V**2) + 0.0455 * V - 1.1349
                EFi_HC = 0.9227 * (V**-0.7830)
                EFi_NOx = 3.8859 * (V**-0.9198)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2006 <= car_birth:
                if V <= 45:
                    EFi_CO = 4.4952 * (V**-0.8461)
                else:
                    EFi_CO = -0.0001 * (V**2) + 0.0229 * V - 0.5701
                EFi_HC = 0.2958 * (V**-0.7830)
                EFi_NOx = 0.4819 * (V**-0.9198)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
        elif (car_type == '승합') and (car_size == '소형'):
            if car_birth <= 1990:
                EFi_CO = 36.169 * (V**-0.7587)
                EFi_HC = 15.607 * (V**-1.0423)
                EFi_NOx = 6.2007 * (V**-0.6781)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 1991 <= car_birth <= 1996:
                EFi_CO = 39.402 * (V**-0.8879)
                EFi_HC = 23.400 * (V**-1.4041)
                EFi_NOx = 7.5244 * (V**-0.7634)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 1997 <= car_birth <= 1999:
                EFi_CO = 77.088 * (V**-1.2078)
                EFi_HC = 18.731 * (V**-1.5356)
                EFi_NOx = 4.4260 * (V**-0.7978)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif car_birth == 2000:
                EFi_CO = 41.669 * (V**-1.2078)
                EFi_HC = 14.190 * (V**-1.5356)
                EFi_NOx = 3.4578 * (V**-0.7978)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2001 <= car_birth <= 2002:
                EFi_CO = 41.669 * (V**-1.2078)
                EFi_HC = 11.920 * (V**-1.5356)
                EFi_NOx = 3.0649 * (V**-0.7978)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2003 <= car_birth <= 2005:
                EFi_CO = 37.573 * (V**-1.2078)
                EFi_HC = 3.1786 * (V**-1.5356)
                EFi_NOx = 1.4931 * (V**-0.7978)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2006 <= car_birth:
                EFi_CO = 32.899 * (V**-1.2078)
                EFi_HC = 2.7387 * (V**-1.5356)
                EFi_NOx = 1.1808 * (V**-0.7978)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
        elif ( (car_type == '승합') and ((car_size == '중형') or (car_size == '대형')) ) or ( (car_type == '화물') and (car_size == '소형') ):
            if car_birth <= 1990:
                EFi_CO = 36.169 * (V**-0.7587)
                EFi_HC = 15.607 * (V**-1.0423)
                EFi_NOx = 6.2007 * (V**-0.6781)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 1991 <= car_birth <= 1996:
                EFi_CO = 39.402 * (V**-0.8879)
                EFi_HC = 23.400 * (V**-1.4041)
                EFi_NOx = 7.5244 * (V**-0.7634)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 1997 <= car_birth <= 1999:
                EFi_CO = 77.088 * (V**-1.2078)
                EFi_HC = 18.731 * (V**-1.5356)
                EFi_NOx = 4.4260 * (V**-0.7978)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2000 <= car_birth <= 2002:
                EFi_CO = 41.669 * (V**-1.2078)
                EFi_HC = 14.190 * (V**-1.5356)
                EFi_NOx = 3.4578 * (V**-0.7978)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2003 <= car_birth <= 2005:
                EFi_CO = 36.578 * (V**-1.2078)
                EFi_HC = 3.0337 * (V**-1.5356)
                EFi_NOx = 2.0104 * (V**-0.7978)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2006 <= car_birth:
                EFi_CO = 14.202 * (V**-1.2078)
                EFi_HC = 1.2233 * (V**-1.5356)
                EFi_NOx = 0.2493 * (V**-0.7978)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
        elif (car_type == '화물') and ( (car_size == '중형') or (car_size == '대형') ):
            if 1993 <= car_birth:
                if V <= 65:
                    EFi_CO = 70
                    EFi_HC = 7
                    EFi_NOx = 4.5
                else:
                    EFi_CO = 55
                    EFi_HC = 3.5
                    EFi_NOx = 7.5
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
    elif fuel == '경유':
        if (car_type == '승용') and (car_size == '경형'):
            if car_birth <= 2005:
                EFi_CO = 0.7392 * (V**-0.7524)
                EFi_HC = 0.0989 * (V**-0.6848)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.0839 * (V**-0.3420)
                EFi_PM2_5 = k * 0.0839 * (V**-0.3420)
            elif 2006 <= car_birth <= 2010:
                EFi_CO = 0.5775 * (V**-0.7524)
                EFi_HC = 0.0825 * (V**-0.6848)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.0420 * (V**-0.3420)
                EFi_PM2_5 = k * 0.0420 * (V**-0.3420)
            elif 2011 <= car_birth <= 2015:
                EFi_CO = 0.5141 * (V**-0.6792)
                EFi_HC = 0.3713 * (V**-0.7513)
                EFi_NOx = 0.0003 * (V**2) - 0.0324 * V + 1.4773
                if V <= 65.4:
                    EFi_PM10 = 0.0225 * (V**-0.7264)
                    EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
                else:
                    EFi_PM10 = 0.0009 * (V**-0.0416)
                    EFi_PM2_5 = k * 0.0009 * (V**-0.0416)
            elif 2016 <= car_birth:
                EFi_CO = 0.4574 * (V**-0.5215)
                EFi_HC = 0.1300 * (V**-0.7265)
                EFi_NOx = 2.7702 * (V**-0.3869)
                if V <= 65.4:
                    EFi_PM10 = 0.0225 * (V**-0.7264)
                    EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
                else:
                    EFi_PM10 = 0.0009 * (V**-0.0416)
                    EFi_PM2_5 = k * 0.0009 * (V**-0.0416)
        elif (car_type == '승용') and (car_size == '소형'):
            if car_birth <= 2004:
                EFi_CO = 5.9672 * (V**-0.9534)
                EFi_HC = 0.6523 * (V**-1.0167)
                EFi_NOx = 24.3491 * (V**-0.7277)
                if V <= 65:
                    EFi_PM10 = 0.3861 * (V**-0.5093)
                    EFi_PM2_5 = k * 0.3861 * (V**-0.5093)
                else:
                    EFi_PM10 = -0.00001 * (V**2) + 0.0026 * V - 0.0618
                    EFi_PM2_5 = k * -0.00001 * (V**2) + 0.0026 * V - 0.0618
            elif car_birth == 2005:
                EFi_CO = 0.7392 * (V**-0.7524)
                EFi_HC = 0.0989 * (V**-0.6848)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.0839 * (V**-0.3420)
                EFi_PM2_5 = k * 0.0839 * (V**-0.3420)
            elif 2006 <= car_birth <= 2010:
                EFi_CO = 0.5775 * (V**-0.7524)
                EFi_HC = 0.0825 * (V**-0.6848)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.0420 * (V**-0.3420)
                EFi_PM2_5 = k * 0.0420 * (V**-0.3420)
            elif 2011 <= car_birth <= 2015:
                EFi_CO = 0.5141 * (V**-0.6792)
                EFi_HC = 0.3713 * (V**-0.7513)
                EFi_NOx = 0.0003 * (V**2) - 0.0324 * V + 1.4773
                if V <= 65.4:
                    EFi_PM10 = 0.0225 * (V**-0.7264)
                    EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
                else:
                    EFi_PM10 = 0.0009 * (V**-0.0416)
                    EFi_PM2_5 = k * 0.0009 * (V**-0.0416)
            elif 2016 <= car_birth:
                EFi_CO = 0.4574 * (V**-0.5215)
                EFi_HC = 0.1300 * (V**-0.7265)
                EFi_NOx = 2.7702 * (V**-0.3869)
                if V <= 65.4:
                    EFi_PM10 = 0.0225 * (V**-0.7264)
                    EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
                else:
                    EFi_PM10 = 0.0009 * (V**-0.0416)
                    EFi_PM2_5 = k * 0.0009 * (V**-0.0416)
        elif (car_type == '승용') and (car_size == '중형'):
            if car_birth <= 2004:
                EFi_CO = 5.9672 * (V**-0.9534)
                EFi_HC = 0.6523 * (V**-1.0167)
                EFi_NOx = 24.3491 * (V**-0.7277)
                if V <= 65:
                    EFi_PM10 = 0.3861 * (V**-0.5093)
                    EFi_PM2_5 = k * 0.3861 * (V**-0.5093)
                else:
                    EFi_PM10 = -0.00001 * (V**2) + 0.0026 * V - 0.0618
                    EFi_PM2_5 = k * -0.00001 * (V**2) + 0.0026 * V - 0.0618
            elif car_birth == 2005:
                EFi_CO = 0.6930 * (V**-0.7524)
                EFi_HC = 0.1865 * (V**-0.6848)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.0723 * (V**-0.3420)
                EFi_PM2_5 = k * 0.0723 * (V**-0.3420)
            elif 2006 <= car_birth <= 2010:
                EFi_CO = 0.5414 * (V**-0.7524)
                EFi_HC = 0.0927 * (V**-0.6848)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.0396 * (V**-0.3420)
                EFi_PM2_5 = k * 0.0396 * (V**-0.3420)
            elif 2011 <= car_birth <= 2015:
                EFi_CO = 0.5141 * (V**-0.6792)
                EFi_HC = 0.3713 * (V**-0.7513)
                EFi_NOx = 0.0003 * (V**2) - 0.0324 * V + 1.4773
                if V <= 65.4:
                    EFi_PM10 = 0.0225 * (V**-0.7264)
                    EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
                else:
                    EFi_PM10 = 0.0009 * (V**-0.0416)
                    EFi_PM2_5 = k * 0.0009 * (V**-0.0416)
            elif 2016 <= car_birth:
                EFi_CO = 0.4574 * (V**-0.5215)
                EFi_HC = 0.1300 * (V**-0.7265)
                EFi_NOx = 2.7702 * (V**-0.3869)
                if V <= 65.4:
                    EFi_PM10 = 0.0225 * (V**-0.7264)
                    EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
                else:
                    EFi_PM10 = 0.0009 * (V**-0.0416)
                    EFi_PM2_5 = k * 0.0009 * (V**-0.0416)
        elif (car_type == '승용') and (car_size == '대형'):
            if car_birth <= 2004:
                EFi_CO = 5.9672 * (V**-0.9534)
                EFi_HC = 0.6523 * (V**-1.0167)
                EFi_NOx = 24.3491 * (V**-0.7277)
                if V <= 65:
                    EFi_PM10 = 0.3861 * (V**-0.5093)
                    EFi_PM2_5 = k * 0.3861 * (V**-0.5093)
                else:
                    EFi_PM10 = -0.00001 * (V**2) + 0.0026 * V - 0.0618
                    EFi_PM2_5 = k * -0.00001 * (V**2) + 0.0026 * V - 0.0618
            elif car_birth == 2005:
                EFi_CO = 0.9609 * (V**-0.7524)
                EFi_HC = 0.1865 * (V**-0.6848)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.0723 * (V**-0.3420)
                EFi_PM2_5 = k * 0.0723 * (V**-0.3420)
            elif 2006 <= car_birth <= 2010:
                EFi_CO = 0.7507 * (V**-0.7524)
                EFi_HC = 0.1554 * (V**-0.6848)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.0361 * (V**-0.3420)
                EFi_PM2_5 = k * 0.0361 * (V**-0.3420)
            elif 2011 <= car_birth <= 2015:
                EFi_CO = 0.5141 * (V**-0.6792)
                EFi_HC = 0.3713 * (V**-0.7513)
                EFi_NOx = 0.0003 * (V**2) - 0.0324 * V + 1.4773
                if V <= 65.4:
                    EFi_PM10 = 0.0225 * (V**-0.7264)
                    EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
                else:
                    EFi_PM10 = 0.0009 * (V**-0.0416)
                    EFi_PM2_5 = k * 0.0009 * (V**-0.0416)
            elif 2016 <= car_birth:
                EFi_CO = 0.4574 * (V**-0.5215)
                EFi_HC = 0.1300 * (V**-0.7265)
                EFi_NOx = 2.7702 * (V**-0.3869)
                if V <= 65.4:
                    EFi_PM10 = 0.0225 * (V**-0.7264)
                    EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
                else:
                    EFi_PM10 = 0.0009 * (V**-0.0416)
                    EFi_PM2_5 = k * 0.0009 * (V**-0.0416)
        elif (car_type == '승합') and (car_size == '소형'):
            if car_birth <= 1990:
                if V <= 65.4:
                    EFi_CO = 3.4539 * (V**-0.4266)
                else:
                    EFi_CO = 0.0051 * V + 0.2212
                EFi_HC = 0.9835 * (V**-0.5096)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 1.1412 * (V**-0.4324)
                EFi_PM2_5 = k * 1.1412 * (V**-0.4324)
            elif 1991 <= car_birth <= 1995:
                if V <= 65.4:
                    EFi_CO = 3.4539 * (V**-0.4266)
                else:
                    EFi_CO = 0.0051 * V + 0.2212
                EFi_HC = 1.6313 * (V**-0.7298)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.5999 * (V**-0.3294)
                EFi_PM2_5 = k * 0.5999 * (V**-0.3294)
            elif 1996 <= car_birth <= 1997:
                if V <= 65.4:
                    EFi_CO = 3.4539 * (V**-0.4266)
                else:
                    EFi_CO = 0.0051 * V + 0.2212
                EFi_HC = 1.1293 * (V**-0.6588)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.6408 * (V**-0.3596)
                EFi_PM2_5 = k * 0.6408 * (V**-0.3596)
            elif 1998 <= car_birth <= 1999:
                EFi_CO = 3.7564 * (V**-0.5175)
                EFi_HC = 1.1293 * (V**-0.6588)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.5168 * (V**-0.3596)
                EFi_PM2_5 = k * 0.5168 * (V**-0.3596)
            elif 2000 <= car_birth <= 2003:
                EFi_CO = 3.7564 * (V**-0.5175)
                EFi_HC = 1.1293 * (V**-0.6588)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.2894 * (V**-0.3596)
                EFi_PM2_5 = k * 0.2894 * (V**-0.3596)
            elif 2004 <= car_birth <= 2007:
                EFi_CO = 3.2797 * (V**-0.8887)
                EFi_HC = 0.1807 * (V**-0.6588)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.2067 * (V**-0.3596)
                EFi_PM2_5 = k * 0.2067 * (V**-0.3596)
            elif 2008 <= car_birth <= 2011:
                if V <= 65.4:
                    EFi_CO = 4.222 * (V**-1.4035)
                else:
                    EFi_CO = 0.011666 * (V**0.09222)
                if V <= 97.3:
                    EFi_HC = 0.829 * (V**-1.0961)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.3111 * (V**-0.5125)
                EFi_PM2_5 = k * 0.3111 * (V**-0.5125)
            elif 2012 <= car_birth:
                if V <= 65.4:
                    EFi_CO = 4.222 * (V**-1.4035)
                else:
                    EFi_CO = 0.011666 * (V**0.09222)
                if V <= 97.3:
                    EFi_HC = 0.829 * (V**-1.0961)
                if V <= 65.4:
                    EFi_NOx = 2.0217 * (V**-0.2645)
                else:
                    EFi_NOx = 0.0271 * (V**0.7596)
                EFi_PM10 = 0.1119 * (V**-0.5125)
                EFi_PM2_5 = k * 0.1119 * (V**-0.5125)
        elif (car_type == '승합') and (car_size == '중형'):
            if car_birth <= 1995:
                EFi_CO = 32.550 * (V**-0.4944)
                EFi_HC = 15.753 * (V**-0.5912)
                if V < 80:
                    EFi_NOx = 40.692 * (V**-0.5590)
                else:
                    EFi_NOx = -0.0023 * (V**2) + 0.5381 * V - 23.590
                EFi_PM10 = 5.4886 * (V**-0.5911)
                EFi_PM2_5 = k * 5.4886 * (V**-0.5911)
            elif 1996 <= car_birth <= 1997:
                EFi_CO = 16.410 * (V**-0.3790)
                EFi_HC = 4.2324 * (V**-0.3926)
                if V < 80:
                    EFi_NOx = 22.804 * (V**-0.4660)
                else:
                    EFi_NOx = -0.0021 * (V**2) + 0.4430 * V - 18.730
                EFi_PM10 = 1.6593 * (V**-0.3935)
                EFi_PM2_5 = k * 1.6593 * (V**-0.3935)
            elif 1998 <= car_birth <= 2000:
                EFi_CO = 16.410 * (V**-0.3790)
                EFi_HC = 4.2324 * (V**-0.3926)
                if V < 80:
                    EFi_NOx = 25.708 * (V**-0.4772)
                else:
                    EFi_NOx = 0.0019 * (V**2) - 0.2628 * V + 12.145
                EFi_PM10 = 1.6593 * (V**-0.3935)
                EFi_PM2_5 = k * 1.6593 * (V**-0.3935)
            elif 2001 <= car_birth <= 2004:
                EFi_CO = 16.378 * (V**-0.5340)
                EFi_HC = 5.8477 * (V**-0.5466)
                if V < 80:
                    EFi_NOx = 25.436 * (V**-0.4656)
                else:
                    EFi_NOx = 0.0008 * (V**2) - 0.0482 * V + 1.8424
                EFi_PM10 = 1.2848 * (V**-0.4715)
                EFi_PM2_5 = k * 1.2848 * (V**-0.4715)
            elif 2005 <= car_birth <= 2007:
                EFi_CO = 15.256 * (V**-0.7448)
                EFi_HC = 2.0502 * (V**-0.6504)
                EFi_NOx = 15.001 * (V**-0.4528)
                EFi_PM10 = 0.2979 * (V**-0.4008)
                EFi_PM2_5 = k * 1.0457 * (V**-0.4527)
            elif 2008 <= car_birth <= 2010:
                EFi_CO = 8.1771 * (V**-0.7725)
                EFi_HC = 1.2991 * (V**-0.6538)
                if V <= 64.7:
                    EFi_NOx = 17.3032 * (V**-0.3660)
                    EFi_PM10 = 0.0539 * (V**-0.5182)
                    EFi_PM2_5 = k * 0.0539 * (V**-0.5182)
                else:
                    EFi_NOx = 0.3259 * (V**0.5773)
                    EFi_PM10 = 2.0000*(10**-4) * V - 7.5600*(10**-3)
                    EFi_PM2_5 = k * 2.0000*(10**-4) * V - 7.5600*(10**-3)
            elif 2011 <= car_birth <= 2014:
                EFi_CO = 4.5201 * (V**-0.7279)
                EFi_HC = 1.6826 * (V**-0.8045)
                if V <= 64.7:
                    EFi_NOx = 17.2485 * (V**-0.4040)
                    EFi_PM10 = 0.0469 * (V**-0.4674)
                    EFi_PM2_5 = k * 0.0469 * (V**-0.4674)
                else:
                    EFi_NOx = 1.1797 * (V**0.2308)
                    EFi_PM10 = 1.6800*(10**-4) * V - 5.1600*(10**-3)
                    EFi_PM2_5 = k * 1.6800*(10**-4) * V - 5.1600*(10**-3)
            elif 2015 <= car_birth:
                EFi_CO = 7.4065 * (V**-0.5995)
                EFi_HC = 2.4562 * (V**-1.3145)
                EFi_NOx = 42.7393 * (V**-1.2949)
                if V <= 64.7:
                    EFi_PM10 = 0.0081 * (V**-0.5182)
                    EFi_PM2_5 = k * 0.0081 * (V**-0.5182)
                else:
                    EFi_PM10 = 3.0000*(10**-5) * V - 0.0011
                    EFi_PM2_5 = k * 3.0000*(10**-5) * V - 0.0011
        elif (car_type == '승합') and (car_size == '대형'):
            if car_birth <= 1995:
                EFi_CO = 28.205 * (V**-0.5337)
                EFi_HC = 6.1146 * (V**-0.4979)
                EFi_NOx = 41.346 * (V**-0.3645)
                EFi_PM10 = 5.2158 * (V**-0.5048)
                EFi_PM2_5 = k * 5.2158 * (V**-0.5048)
            elif 1996 <= car_birth <= 1997:
                EFi_CO = 23.205 * (V**-0.5425)
                EFi_HC = 6.5657 * (V**-0.5431)
                EFi_NOx = 42.1379 * (V**-0.3786)
                EFi_PM10 = 2.4911 * (V**-0.4149)
                EFi_PM2_5 = k * 2.4911 * (V**-0.4149)
            elif 1998 <= car_birth <= 2000:
                EFi_CO = 23.205 * (V**-0.5425)
                EFi_HC = 6.5657 * (V**-0.5431)
                EFi_NOx = 42.1379 * (V**-0.3786)
                EFi_PM10 = 1.4432 * (V**-0.3870)
                EFi_PM2_5 = k * 1.4432 * (V**-0.3870)
            elif car_birth == 2001:
                EFi_CO = 21.348 * (V**-0.5806)
                EFi_HC = 6.6390 * (V**-0.5760)
                EFi_NOx = 36.7191 * (V**-0.3548)
                EFi_PM10 = 0.9375 * (V**-0.3910)
                EFi_PM2_5 = k * 0.9375 * (V**-0.3910)
            elif 2002 <= car_birth <= 2004:
                EFi_CO = 21.348 * (V**-0.5806)
                EFi_HC = 6.6390 * (V**-0.5760)
                EFi_NOx = 36.7191 * (V**-0.3548)
                EFi_PM10 = 1.1507 * (V**-0.4804)
                EFi_PM2_5 = k * 1.1507 * (V**-0.4804)
            elif 2005 <= car_birth <= 2007:
                EFi_CO = 9.6452 * (V**-0.5291)
                EFi_HC = 3.2339 * (V**-0.7436)
                EFi_NOx = 30.5870 * (V**-0.3548)
                if V <= 80:
                    EFi_PM10 = 0.4657 * (V**-0.5634)
                    EFi_PM2_5 = k * 0.4657 * (V**-0.5634)
                else:
                    EFi_PM10 = 0.0014 * (V**0.7970)
                    EFi_PM2_5 = k * 0.0014 * (V**0.7970)
            elif 2008 <= car_birth <= 2010:
                EFi_CO = 6.8493 * (V**-0.6506)
                EFi_HC = 1.7177 * (V**-0.6781)
                EFi_NOx = 40.7564 * (V**-0.4757)
                EFi_PM10 = 0.2418 * (V**-0.4727)
                EFi_PM2_5 = k * 0.2418 * (V**-0.4727)
            elif 2011 <= car_birth <= 2014:
                EFi_CO = 5.4607 * (V**-0.2990)
                EFi_HC = 0.8863 * (V**-0.6933)
                EFi_NOx = 40.3729 * (V**-0.5386)
                EFi_PM10 = 0.2125 * (V**-0.4650)
                EFi_PM2_5 = k * 0.2125 * (V**-0.4650)
            elif 2015 <= car_birth:
                EFi_CO = 11.4415 * (V**-0.8036)
                EFi_HC = 0.6774 * (V**-0.8321)
                EFi_NOx = 112.1229 * (V**-1.6393)
                EFi_PM10 = 0.0363 * (V**-0.4727)
                EFi_PM2_5 = k * 0.0363 * (V**-0.4727)
        elif (car_type == '화물') and (car_size == '소형'):
            if car_birth <= 1990:
                EFi_CO = 4.5854 * (V**-0.3613)
                EFi_HC = 0.4840 * (V**-0.2756)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.8117 * (V**-0.4071)
                EFi_PM2_5 = k * 0.8117 * (V**-0.4071)
            elif 1991 <= car_birth <= 1995:
                EFi_CO = 3.4774 * (V**-0.3483)
                EFi_HC = 0.4844 * (V**-0.3288)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.6188 * (V**-0.4540)
                EFi_PM2_5 = k * 0.6188 * (V**-0.4540)
            elif 1996 <= car_birth <= 1997:
                EFi_CO = 3.3934 * (V**-0.3837)
                EFi_HC = 0.4955 * (V**-0.3393)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.7037 * (V**-0.5357)
                EFi_PM2_5 = k * 0.7037 * (V**-0.5357)
            elif 1998 <= car_birth <= 1999:
                EFi_CO = 4.0896 * (V**-0.6083)
                EFi_HC = 0.6122 * (V**-0.5684)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.6157 * (V**-0.5357)
                EFi_PM2_5 = k * 0.6157 * (V**-0.5357)
            elif 2000 <= car_birth <= 2003:
                EFi_CO = 4.0896 * (V**-0.6083)
                EFi_HC = 0.6122 * (V**-0.5684)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.4838 * (V**-0.5357)
                EFi_PM2_5 = k * 0.4838 * (V**-0.5357)
            elif 2004 <= car_birth <= 2007:
                EFi_CO = 3.2797 * (V**-0.8887)
                EFi_HC = 0.1807 * (V**-0.6588)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.2067 * (V**-0.3596)
                EFi_PM2_5 = k * 0.2067 * (V**-0.3596)
            elif 2008 <= car_birth <= 2011:
                if V <= 65.4:
                    EFi_CO = 4.222 * (V**-1.4035)
                else:
                    EFi_CO = 0.01166 * (V**0.09222)
                if V <= 97.3:
                    EFi_HC = 0.829 * (V**-1.0961)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.3111 * (V**-0.5125)
                EFi_PM2_5 = k * 0.3111 * (V**-0.5125)
            elif 2012 <= car_birth <= 2016:
                if V <= 65.4:
                    EFi_CO = 4.222 * (V**-1.4035)
                    EFi_NOx = 2.0217 * (V**-0.2645)
                else:
                    EFi_CO = 0.01166 * (V**0.09222)
                    EFi_NOx = 0.0271 * (V**0.7596)
                if V <= 97.3:
                    EFi_HC = 0.829 * (V**-1.0961)
                EFi_PM10 = 0.1119 * (V**-0.5125)
                EFi_PM2_5 = k * 0.1119 * (V**-0.5125)
        elif ( (car_type == '화물') and (car_size == '중형') ) | (car_type == '특수'):
            if car_birth <= 1995:
                EFi_CO = 16.769 * (V**-0.3772)
                EFi_HC = 6.7755 * (V**-0.5003)
                EFi_NOx = 24.915 * (V**-0.3942)
                EFi_PM10 = 3.6772 * (V**-0.5514)
                EFi_PM2_5 = k * 3.6772 * (V**-0.5514)
            elif 1996 <= car_birth <= 2000:
                EFi_CO = 21.057 * (V**-0.4958)
                EFi_HC = 6.7532 * (V**-0.5711)
                EFi_NOx = 25.022 * (V**-0.4240)
                EFi_PM10 = 3.5285 * (V**-0.5962)
                EFi_PM2_5 = k * 3.5285 * (V**-0.5962)
            elif car_birth == 2001:
                EFi_CO = 25.501 * (V**-0.6100)
                EFi_HC = 6.8738 * (V**-0.5913)
                if V <= 64.7:
                    EFi_NOx = 17.3032 * (V**-0.3660)
                else:
                    EFi_NOx = 0.3259 * (V**0.5773)
                EFi_PM10 = 1.4444 * (V**-0.4824)
                EFi_PM2_5 = k * 1.4444 * (V**-0.4824)
            elif 2002 <= car_birth <= 2004:
                EFi_CO = 25.501 * (V**-0.6100)
                EFi_HC = 6.8738 * (V**-0.5913)
                if V <= 64.7:
                    EFi_NOx = 17.3032 * (V**-0.3660)
                else:
                    EFi_NOx = 0.3259 * (V**0.5773)
                EFi_PM10 = 1.0432 * (V**-0.4992)
                EFi_PM2_5 = k * 1.0432 * (V**-0.4992)
            elif 2005 <= car_birth <= 2007:
                EFi_CO = 15.256 * (V**-0.7448)
                EFi_HC = 2.0502 * (V**-0.6504)
                if V <= 64.7:
                    EFi_NOx = 17.3032 * (V**-0.3660)
                else:
                    EFi_NOx = 0.3259 * (V**0.5773)
                EFi_PM10 = 0.2979 * (V**-0.4008)
                EFi_PM2_5 = k * 0.2979 * (V**-0.4008)
            elif 2008 <= car_birth <= 2010:
                EFi_CO = 8.1771 * (V**-0.7725)
                EFi_HC = 1.2991 * (V**-0.6538)
                if V <= 64.7:
                    EFi_NOx = 17.3032 * (V**-0.3660)
                    EFi_PM10 = 0.0539 * (V**-0.5182)
                    EFi_PM2_5 = k * 0.0539 * (V**-0.5182)
                else:
                    EFi_NOx = 0.3259 * (V**0.5773)
                    EFi_PM10 = 0.0002 * V - 0.00756
                    EFi_PM2_5 = k * 0.0002 * V - 0.00756
            elif 2011 <= car_birth <= 2014:
                EFi_CO = 4.5201 * (V**-0.7279)
                EFi_HC = 1.6826 * (V**-0.8045)
                if V <= 64.7:
                    EFi_NOx = 17.2485 * (V**-0.4040)
                    EFi_PM10 = 0.0469 * (V**-0.4674)
                    EFi_PM2_5 = k * 0.0469 * (V**-0.4674)
                else:
                    EFi_NOx = 1.1797 * (V**0.2308)
                    EFi_PM10 = 0.000168 * V - 0.00516
                    EFi_PM2_5 = k * 0.000168 * V - 0.00516
            elif 2015 <= car_birth:
                EFi_CO = 7.4065 * (V**-0.5995)
                EFi_HC = 2.4562 * (V**-1.3145)
                EFi_NOx = 42.7393 * (V**-1.2949)
                if V <= 64.7:
                    EFi_PM10 = 0.0081 * (V**-0.5182)
                    EFi_PM2_5 = k * 0.0081 * (V**-0.5182)
                else:
                    EFi_PM10 = 3.0000 * (10**-5) * V - 0.0011
                    EFi_PM2_5 = k * 3.0000 * (10**-5) * V - 0.0011
        elif (car_type == '화물') and (car_size == '대형'):
            if car_birth <= 1995:
                EFi_CO = 30.402 * (V**-0.4685)
                EFi_HC = 15.75 * (V**-0.582)
                EFi_NOx = 117.49 * (V**-0.365)
                EFi_PM10 = 7.6212 * (V**-0.4183)
                EFi_PM2_5 = k * 7.6212 * (V**-0.4183)
            elif 1996 <= car_birth <= 1997:
                EFi_CO = 18.101 * (V**-0.3454)
                EFi_HC = 10.301 * (V**-0.5856)
                EFi_NOx = 94.319 * (V**-0.4061)
                EFi_PM10 = 6.0264 * (V**-0.4627)
                EFi_PM2_5 = k * 6.0264 * (V**-0.4627)
            elif 1998 <= car_birth <= 2000:
                EFi_CO = 18.101 * (V**-0.3454)
                EFi_HC = 10.301 * (V**-0.5856)
                EFi_NOx = 94.319 * (V**-0.4061)
                EFi_PM10 = 4.873 * (V**-0.4382)
                EFi_PM2_5 = k * 4.873 * (V**-0.4382)
            elif 2001 <= car_birth <= 2004:
                EFi_CO = 28.399 * (V**-0.5999)
                EFi_HC = 10.031 * (V**-0.5828)
                EFi_NOx = 85.301 * (V**-0.4023)
                EFi_PM10 = 3.7541 * (V**-0.4055)
                EFi_PM2_5 = k * 3.7541 * (V**-0.4055)
            elif 2005 <= car_birth <= 2007:
                EFi_CO = 52.136 * (V**-0.8618)
                EFi_HC = 3.7878 * (V**-0.5425)
                EFi_NOx = 107.5 * (V**-0.5679)
                EFi_PM10 = 2.6847 * (V**-0.6112)
                EFi_PM2_5 = k * 2.6847 * (V**-0.6112)
            elif 2008 <= car_birth <= 2010:
                EFi_CO = 6.8493 * (V**-0.6506)
                EFi_HC = 1.7177 * (V**-0.6781)
                EFi_NOx = 40.7564 * (V**-0.4757)
                EFi_PM10 = 0.2418 * (V**-0.4727)
                EFi_PM2_5 = k * 0.2418 * (V**-0.4727)
            elif 2011 <= car_birth <= 2014:
                EFi_CO = 5.4607 * (V**-0.2990)
                EFi_HC = 0.8863 * (V**-0.6933)
                EFi_NOx = 40.3729 * (V**-0.5386)
                EFi_PM10 = 0.2125 * (V**-0.4650)
                EFi_PM2_5 = k * 0.2125 * (V**-0.4650)
            elif 2015 <= car_birth:
                EFi_CO = 4.3762 * (V**-0.4550)
                EFi_HC = 0.3627 * (V**-0.7071)
                EFi_NOx = 18.0405 * (V**-1.0986)
                EFi_PM10 = 0.0363 * (V**-0.4727)
                EFi_PM2_5 = k * 0.0363 * (V**-0.4727)
    elif fuel == 'LPG(액화석유가스)':
        if (car_type == '승용') and (car_size == '경형'):
            if car_birth <= 1996:
                if V <= 45:
                    EFi_CO = 22.498 * (V**-0.6579)
                else:
                    EFi_CO = 0.0006 * (V**2) + 0.0004 * V + 0.8272
                EFi_HC = 12.961 * (V**-0.8364)
                EFi_NOx = 4.0131 * (V**-0.3236)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 1997 <= car_birth <= 1999:
                if V <= 45:
                    EFi_CO = 19.887 * (V**-0.8461)
                else:
                    EFi_CO = -0.0004 * (V**2) + 0.0911 * V - 2.2698
                EFi_HC = 2.2714 * (V**-0.7830)
                EFi_NOx = 1.8528 * (V**-0.3889)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 2000 <= car_birth <= 2002:
                if V <= 45:
                    EFi_CO = 19.887 * (V**-0.8461)
                else:
                    EFi_CO = -0.0004 * (V**2) + 0.0911 * V - 2.2698
                EFi_HC = 2.2714 * (V**-0.7830)
                EFi_NOx = 5.8289 * (V**-0.9198)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 2003 <= car_birth <= 2005:
                if V <= 45:
                    EFi_CO = 19.887 * (V**-0.8461)
                else:
                    EFi_CO = -0.0004 * (V**2) + 0.0911 * V - 2.2698
                EFi_HC = 1.1073 * (V**-0.7830)
                EFi_NOx = 5.8289 * (V**-0.9198)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 2006 <= car_birth <= 2007:
                if V <= 45:
                    EFi_CO = 8.9904 * (V**-0.8461)
                else:
                    EFi_CO = -0.0002 * (V**2) + 0.0457 * V - 1.1403
                EFi_HC = 0.3549 * (V**-0.7830)
                EFi_NOx = 0.7228 * (V**-0.9198)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif car_birth == 2008:
                if V <= 79.6:
                    EFi_CO = 0.7693 * (V**-0.7666)
                    EFi_HC = 0.1063 * (V**-1.0745)
                else:
                    EFi_CO = 5.0000 * (10**-16) * (V**7.2766)
                    EFi_HC = 1.0000 * (10**-15) * (V**6.2696)
                EFi_NOx = -4.0000 * (10**-6) * (V**2) + 6.0000 * (10**-4) * V + 5.5000 * (10**-3)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 2009 <= car_birth <= 2011:
                if V <= 79.6:
                    EFi_CO = 0.7059 * (V**-0.7666)
                    EFi_HC = 0.0974 * (V**-1.0745)
                else:
                    EFi_CO = 4.5878 * (10**-16) * (V**7.2766)
                    EFi_HC = 9.1667 * (10**-16) * (V**6.2696)
                EFi_NOx = -3.7333 * (10**-6) * (V**2) + 5.6000 * (10**-4) * V + 5.1333 * (10**-3)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 2012 <= car_birth <= 2013:
                if V <= 79.6:
                    EFi_CO = 0.6830 * (V**-0.7666)
                    EFi_HC = 0.0943 * (V**-1.0745)
                else:
                    EFi_CO = 4.4393 * (10**-16) * (V**7.2766)
                    EFi_HC = 8.8667 * (10**-16) * (V**6.2696)
                EFi_NOx = -3.6373 * (10**-6) * (V**2) + 5.4560 * (10**-4) * V + 5.0013 * (10**-3)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif car_birth == 2014:
                if V <= 79.6:
                    EFi_CO = 0.6602 * (V**-0.7666)
                    EFi_HC = 0.0911 * (V**-1.0745)
                else:
                    EFi_CO = 4.2909 * (10**-16) * (V**7.2766)
                    EFi_HC = 8.5667 * (10**-16) * (V**6.2696)
                EFi_NOx = -3.5413 * (10**-6) * (V**2) + 5.3120 * (10**-4) * V + 4.8693 * (10**-3)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 2015 <= car_birth:
                if V <= 79.6:
                    EFi_CO = 0.6374 * (V**-0.7666)
                    EFi_HC = 0.0879 * (V**-1.0745)
                else:
                    EFi_CO = 4.1425 * (10**-16) * (V**7.2766)
                    EFi_HC = 8.2667 * (10**-16) * (V**6.2696)
                EFi_NOx = -3.4453 * (10**-6) * (V**2) + 5.1680 * (10**-4) * V + 4.7373 * (10**-3)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
        elif (car_type == '승용') and (car_size == '소형'):
            if car_birth <= 1990:
                EFi_CO = 72.338 * (V**-0.7587)
                EFi_HC = 26.532 * (V**-1.0423)
                EFi_NOx = 9.3011 * (V**-0.6781)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 1991 <= car_birth == 1996:
                EFi_CO = 72.338 * (V**-0.7587)
                EFi_HC = 101.79 * (V**-1.6823)
                EFi_NOx = 11.287 * (V**-0.7634)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 1997 <= car_birth <= 1999:
                EFi_CO = 44.956 * (V**-1.0085)
                EFi_HC = 11.173 * (V**-1.3927)
                EFi_NOx = 7.5371 * (V**-0.7864)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 2000 <= car_birth <= 2002:
                EFi_CO = 44.956 * (V**-1.0085)
                EFi_HC = 11.173 * (V**-1.3927)
                EFi_NOx = 4.7108 * (V**-0.7864)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 2003 <= car_birth <= 2005:
                EFi_CO = 44.956 * (V**-1.0085)
                EFi_HC = 3.2821 * (V**-1.3927)
                EFi_NOx = 4.7108 * (V**-0.7864)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 2006 <= car_birth <= 2007:
                EFi_CO = 39.362 * (V**-1.0085)
                EFi_HC = 2.8981 * (V**-1.3927)
                EFi_NOx = 1.8419 * (V**-0.7864)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif car_birth == 2008:
                if V <= 79.6:
                    EFi_CO = 0.7693 * (V**-0.7666)
                    EFi_HC = 0.1063 * (V**-1.0745)
                else:
                    EFi_CO = 5.0000 * (10**-16) * (V**7.2766)
                    EFi_HC = 1.0000 * (10**-15) * (V**6.2696)
                EFi_NOx = -4.0000 * (10**-6) * (V**2) + 6.0000 * (10**-4) * V + 5.5000 * (10**-3)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 2009 <= car_birth <= 2011:
                if V <= 79.6:
                    EFi_CO = 0.7059 * (V**-0.7666)
                    EFi_HC = 0.0974 * (V**-1.0745)
                else:
                    EFi_CO = 4.5878 * (10**-16) * (V**7.2766)
                    EFi_HC = 9.1667 * (10**-16) * (V**6.2696)
                EFi_NOx = -3.7333 * (10**-6) * (V**2) + 5.6000 * (10**-4) * V + 5.1333 * (10**-3)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 2012 <= car_birth <= 2013:
                if V <= 79.6:
                    EFi_CO = 0.6830 * (V**-0.7666)
                    EFi_HC = 0.0943 * (V**-1.0745)
                else:
                    EFi_CO = 4.4393 * (10**-16) * (V**7.2766)
                    EFi_HC = 8.8667 * (10**-16) * (V**6.2696)
                EFi_NOx = -3.6373 * (10**-6) * (V**2) + 5.4560 * (10**-4) * V + 5.0013 * (10**-3)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif car_birth == 2014:
                if V <= 79.6:
                    EFi_CO = 0.6602 * (V**-0.7666)
                    EFi_HC = 0.0911 * (V**-1.0745)
                else:
                    EFi_CO = 4.2909 * (10**-16) * (V**7.2766)
                    EFi_HC = 8.5667 * (10**-16) * (V**6.2696)
                EFi_NOx = -3.5413 * (10**-6) * (V**2) + 5.3120 * (10**-4) * V + 4.8693 * (10**-3)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 2015 <= car_birth:
                if V <= 79.6:
                    EFi_CO = 0.6374 * (V**-0.7666)
                    EFi_HC = 0.0879 * (V**-1.0745)
                else:
                    EFi_CO = 4.1425 * (10**-16) * (V**7.2766)
                    EFi_HC = 8.2667 * (10**-16) * (V**6.2696)
                EFi_NOx = -3.4453 * (10**-6) * (V**2) + 5.1680 * (10**-4) * V + 4.7373 * (10**-3)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
        elif ( (car_type == '승용') and ( (car_size == '중형') or (car_size == '대형') ) or ( (car_type == '특수') and (car_size == '중형') ) ): # 최 확인(2023.04.20) : 특수 중형 -> 승용 중형 조건으로 계산
            if car_birth <= 1990:
                EFi_CO = 72.338 * (V**-0.7587)
                EFi_HC = 26.532 * (V**-1.0423)
                EFi_NOx = 9.3011 * (V**-0.6781)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 1991 <= car_birth == 1996:
                EFi_CO = 29.825 * (V**-0.6771)
                EFi_HC = 26.520 * (V**-1.4041)
                EFi_NOx = 11.287 * (V**-0.7634)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 1997 <= car_birth <= 1999:
                EFi_CO = 29.825 * (V**-0.6771)
                EFi_HC = 4.7595 * (V**-0.9512)
                EFi_NOx = 12.562 * (V**-0.8606)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 2000 <= car_birth <= 2005:
                EFi_CO = 17.829 * (V**-0.6778)
                EFi_HC = 6.3668 * (V**-1.2849)
                EFi_NOx = 8.8952 * (V**-0.8119)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 2006 <= car_birth <= 2007:
                EFi_CO = 73.022 * (V**-1.2078)
                EFi_HC = 4.4166 * (V**-1.5356)
                EFi_NOx = 2.0280 * (V**-0.7978)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif car_birth == 2008:
                if V <= 79.6:
                    EFi_CO = 0.7693 * (V**-0.7666)
                    EFi_HC = 0.1063 * (V**-1.0745)
                else:
                    EFi_CO = 5.0000 * (10**-16) * (V**7.2766)
                    EFi_HC = 1.0000 * (10**-15) * (V**6.2696)
                EFi_NOx = -4.0000 * (10**-6) * (V**2) + 6.0000 * (10**-4) * V + 5.5000 * (10**-3)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 2009 <= car_birth <= 2011:
                if V <= 79.6:
                    EFi_CO = 0.7059 * (V**-0.7666)
                    EFi_HC = 0.0974 * (V**-1.0745)
                else:
                    EFi_CO = 4.5878 * (10**-16) * (V**7.2766)
                    EFi_HC = 9.1667 * (10**-16) * (V**6.2696)
                EFi_NOx = -3.7333 * (10**-6) * (V**2) + 5.6000 * (10**-4) * V + 5.1333 * (10**-3)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 2012 <= car_birth <= 2013:
                if V <= 79.6:
                    EFi_CO = 0.6830 * (V**-0.7666)
                    EFi_HC = 0.0943 * (V**-1.0745)
                else:
                    EFi_CO = 4.4393 * (10**-16) * (V**7.2766)
                    EFi_HC = 8.8667 * (10**-16) * (V**6.2696)
                EFi_NOx = -3.6373 * (10**-6) * (V**2) + 5.4560 * (10**-4) * V + 5.0013 * (10**-3)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif car_birth == 2014:
                if V <= 79.6:
                    EFi_CO = 0.6602 * (V**-0.7666)
                    EFi_HC = 0.0911 * (V**-1.0745)
                else:
                    EFi_CO = 4.2909 * (10**-16) * (V**7.2766)
                    EFi_HC = 8.5667 * (10**-16) * (V**6.2696)
                EFi_NOx = -3.5413 * (10**-6) * (V**2) + 5.3120 * (10**-4) * V + 4.8693 * (10**-3)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 2015 <= car_birth:
                if V <= 79.6:
                    EFi_CO = 0.6374 * (V**-0.7666)
                    EFi_HC = 0.0879 * (V**-1.0745)
                else:
                    EFi_CO = 4.1425 * (10**-16) * (V**7.2766)
                    EFi_HC = 8.2667 * (10**-16) * (V**6.2696)
                EFi_NOx = -3.4453 * (10**-6) * (V**2) + 5.1680 * (10**-4) * V + 4.7373 * (10**-3)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
        elif ( (car_type == '승합') and (car_size == '경형') ) or ( (car_type == '화물') and (car_size == '경형') ):
            if car_birth <= 1996:
                if V <= 45:
                    EFi_CO = 22.498 * (V**-0.6579)
                else:
                    EFi_CO = 0.0006 * (V**2) + 0.0004 * V + 0.8272
                EFi_HC = 12.961 * (V**-0.8364)
                EFi_NOx = 4.0131 * (V**-0.3236)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 1997 <= car_birth == 1999:
                if V <= 45:
                    EFi_CO = 19.887 * (V**-0.8461)
                else:
                    EFi_CO = -0.0004 * (V**2) + 0.0911 * V - 2.2698
                EFi_HC = 2.2714 * (V**-0.7830)
                EFi_NOx = 1.8528 * (V**-0.3889)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 2000 <= car_birth <= 2002:
                if V <= 45:
                    EFi_CO = 19.887 * (V**-0.8461)
                else:
                    EFi_CO = -0.0004 * (V**2) + 0.0911 * V - 2.2698
                EFi_HC = 2.2714 * (V**-0.7830)
                EFi_NOx = 5.8289 * (V**-0.9198)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 2003 <= car_birth <= 2005:
                if V <= 45:
                    EFi_CO = 19.887 * (V**-0.8461)
                else:
                    EFi_CO = -0.0004 * (V**2) + 0.0911 * V - 2.2698
                EFi_HC = 1.1073 * (V**-0.7830)
                EFi_NOx = 5.8289 * (V**-0.9198)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 2006 <= car_birth:
                if V <= 45:
                    EFi_CO = 8.9904 * (V**-0.8461)
                else:
                    EFi_CO = -0.0002 * (V**2) + 0.0457 * V - 1.1403
                EFi_HC = 0.3549 * (V**-0.7830)
                EFi_NOx = 0.7228 * (V**-0.9198)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
        elif (car_type == '승합') and (car_size == '소형'):
            if car_birth <= 1990:
                EFi_CO = 72.338 * (V**-0.7587)
                EFi_HC = 26.532 * (V**-1.0423)
                EFi_NOx = 9.3011 * (V**-0.6781)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 1991 <= car_birth == 1996:
                EFi_CO = 29.825 * (V**-0.6771)
                EFi_HC = 26.520 * (V**-1.4041)
                EFi_NOx = 11.287 * (V**-0.7634)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 1997 <= car_birth <= 1999:
                EFi_CO = 29.825 * (V**-0.6771)
                EFi_HC = 4.7595 * (V**-0.9512)
                EFi_NOx = 12.562 * (V**-0.8606)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 2000 <= car_birth <= 2005:
                EFi_CO = 17.829 * (V**-0.6778)
                EFi_HC = 6.3668 * (V**-1.2849)
                EFi_NOx = 8.8952 * (V**-0.8119)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 2006 <= car_birth:
                if V <= 65.4:
                    EFi_CO = 6.1701 * (V**-1.0719)
                    EFi_HC = 2.3782 * (V**-1.9979)
                else:
                    EFi_CO = 3.0000 * (10**-10) * (V**4.5809)
                    EFi_HC = 5.0000 * (10**-10) * (V**3.4895)
                if V <= 97.3:
                    EFi_NOx = 0.4758 * (V**-1.0665)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
        # 최 확인(2023.04.20) : 화물 중형 -> 승합 중형 조건, 화물 대형 -> 승합 대형 조건으로 계산
        elif ( (car_type == '승합') and ( (car_size == '중형') or (car_size == '대형') ) ) or ( (car_type == '화물') and ((car_size == '소형') or (car_size == '중형') or (car_size == '대형')) ):
            if car_birth <= 1990:
                EFi_CO = 72.338 * (V**-0.7587)
                EFi_HC = 26.532 * (V**-1.0423)
                EFi_NOx = 9.3011 * (V**-0.6781)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 1991 <= car_birth == 1996:
                EFi_CO = 29.825 * (V**-0.6771)
                EFi_HC = 26.520 * (V**-1.4041)
                EFi_NOx = 11.287 * (V**-0.7634)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 1997 <= car_birth <= 1999:
                EFi_CO = 29.825 * (V**-0.6771)
                EFi_HC = 4.7595 * (V**-0.9512)
                EFi_NOx = 12.562 * (V**-0.8606)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 2000 <= car_birth <= 2005:
                EFi_CO = 17.829 * (V**-0.6778)
                EFi_HC = 6.3668 * (V**-1.2849)
                EFi_NOx = 8.8952 * (V**-0.8119)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 2006 <= car_birth:
                if V <= 65.4:
                    EFi_CO = 16.491 * (V**-1.4713)
                    EFi_HC = 1.9218 * (V**-1.8418)
                else:
                    EFi_CO = 3.0000 * (10**-13) * (V**6.0619)
                    EFi_HC = 3.0000 * (10**-9) * (V**3.0639)
                if V <= 97.3:
                    EFi_NOx = 0.1246 * (V**-0.441)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
    else:
        # 값 고민 : 1
        # EFi_CO, EFi_HC, EFi_NOx, EFi_PM10, EFi_PM2_5 = np.nan, np.nan, np.nan, np.nan, np.nan
        EFi_CO, EFi_HC, EFi_NOx, EFi_PM10, EFi_PM2_5 = 1, 1, 1, 1, 1

    # 배출계수(EFi) 리스트에 저장
    EFi_CO_list.append(EFi_CO)
    EFi_HC_list.append(EFi_HC)
    EFi_NOx_list.append(EFi_NOx)
    EFi_PM10_list.append(EFi_PM10)
    EFi_PM2_5_list.append(EFi_PM2_5)

100%|██████████| 25117108/25117108 [01:48<00:00, 231884.90it/s]


In [250]:
df2.shape[0]

25117108

In [251]:
len(EFi_CO_list), len(EFi_HC_list), len(EFi_NOx_list), len(EFi_PM10_list), len(EFi_PM2_5_list)

(25117108, 25117108, 25117108, 25117108, 25117108)

In [252]:
df2['EFi_CO'] = EFi_CO_list
df2['EFi_HC'] = EFi_HC_list
df2['EFi_NOx'] = EFi_NOx_list
df2['EFi_PM10'] = EFi_PM10_list
df2['EFi_PM2_5'] = EFi_PM2_5_list

In [253]:
check_EFi_col = ['EFi_CO', 'EFi_HC', 'EFi_NOx', 'EFi_PM10', 'EFi_PM2_5']

### 배출량 계산

In [254]:
DF_col + check_DF_col + check_R_col

['fuel',
 '차종',
 '차종유형',
 '차량연식',
 '차령',
 'DF_CO',
 'DF_HC',
 'DF_PM',
 'DF_NOx',
 'R_CO',
 'R_HC',
 'R_PM']

In [255]:
df2[DF_col + check_DF_col + check_R_col + check_EFi_col].head()

,fuel,차종,차종유형,차량연식,차령,DF_CO,DF_HC,DF_PM,DF_NOx,R_CO,R_HC,R_PM,EFi_CO,EFi_HC,EFi_NOx,EFi_PM10,EFi_PM2_5
0,휘발유,승용,소형,1976,47,2.00,2.00,2.00,2.00,0.00,0.00,0.00,17.38,2.12,1.25,0.00,0.00
1,휘발유,승용,경형,1968,55,2.00,2.00,2.00,2.00,0.00,0.00,0.00,1.10,0.27,0.74,0.00,0.00
2,휘발유,승용,소형,1976,47,2.00,2.00,2.00,2.00,0.00,0.00,0.00,17.38,2.12,1.25,0.00,0.00
3,경유,화물,중형,1971,52,1.50,1.50,1.50,1.20,0.00,0.00,0.00,3.72,0.92,5.17,0.41,0.37
4,휘발유,승용,소형,1986,37,2.00,2.00,2.00,2.00,0.00,0.00,0.00,17.38,2.12,1.25,0.00,0.00


In [256]:
# E = VKT * (EFi / 1000) * DF * (1 - R / 100)
df2['E_HOT_CO'] = df2['VKT'] * (df2['EFi_CO'] / 1000) * df2['DF_CO'] * (1 - df2['R_CO'] / 100)
df2['E_HOT_HC'] = df2['VKT'] * (df2['EFi_HC'] / 1000) * df2['DF_HC'] * (1 - df2['R_HC'] / 100)
df2['E_HOT_NOx'] = df2['VKT'] * (df2['EFi_NOx'] / 1000) * df2['DF_NOx'] * (1 - 0 / 100)
df2['E_HOT_PM10'] = df2['VKT'] * (df2['EFi_PM10'] / 1000) * df2['DF_PM'] * (1 - df2['R_PM'] / 100)
df2['E_HOT_PM2_5'] = df2['VKT'] * (df2['EFi_PM2_5'] / 1000) * df2['DF_PM'] * (1 - df2['R_PM'] / 100)

In [257]:
check_E_HOT_col = ['E_HOT_CO', 'E_HOT_HC', 'E_HOT_NOx', 'E_HOT_PM10', 'E_HOT_PM2_5']

In [258]:
df2[DF_col + check_DF_col + check_R_col + check_EFi_col + check_E_HOT_col].head()

,fuel,차종,차종유형,차량연식,차령,DF_CO,DF_HC,DF_PM,DF_NOx,R_CO,...,EFi_CO,EFi_HC,EFi_NOx,EFi_PM10,EFi_PM2_5,E_HOT_CO,E_HOT_HC,E_HOT_NOx,E_HOT_PM10,E_HOT_PM2_5
0,휘발유,승용,소형,1976,47,2.00,2.00,2.00,2.00,0.00,...,17.38,2.12,1.25,0.00,0.00,405.90,49.49,29.31,0.01,0.01
1,휘발유,승용,경형,1968,55,2.00,2.00,2.00,2.00,0.00,...,1.10,0.27,0.74,0.00,0.00,30.01,7.37,20.02,0.01,0.01
2,휘발유,승용,소형,1976,47,2.00,2.00,2.00,2.00,0.00,...,17.38,2.12,1.25,0.00,0.00,488.35,59.54,35.26,0.01,0.01
3,경유,화물,중형,1971,52,1.50,1.50,1.50,1.20,0.00,...,3.72,0.92,5.17,0.41,0.37,97.81,24.18,108.63,10.70,9.85
4,휘발유,승용,소형,1986,37,2.00,2.00,2.00,2.00,0.00,...,17.38,2.12,1.25,0.00,0.00,483.28,58.92,34.89,0.01,0.01


In [259]:
df2.shape

(25117108, 34)

## 자동차-엔진 미가열(Cold-start) 배출

### 배출량 식
$$E_{COLD :i, j} = \beta_{i, j} \times N_j \times M_j \times e^{HOT}_j \times (e^{COLD} / e^{HOT} \vert_{i, j} - 1)$$

- $E_{COLD :i, j}$ : 차종 $j$에서 배출되는 오염물질 $i$의 엔진미가열 배출량
- $\beta_{i, j}$ : 차종 $j$의 엔진미가열 상태의 주행거리 분율
- $N_j$ : 차종 $j$의 수
- $M_j$ : 차종 $j$의 주행거리
- $e^{HOT}_j$ : 차종 $j$의 엔진가열 상태에서의 배출계수
- $e^{COLD} / e^{HOT} \vert_{i, j}$ : 차종 $j$의 엔진가열상태 대비 엔진미가열 상태에서의 배출 비율

### 베타($\beta$)
- 1회 평균주행거리(1 trip length)와 대기온도, 자동차 이용 패턴을 고려
    - 1회 평균주행거리 정보는 모든 차종에 대하여 각각 적용하는 방안은 수집자료의 한계로 국내 연구결과를 바탕으로 수도권 지역 승용차의 1회 평균 주행거리 12.35km를 적용
    - 이때 과대산정 가능성을 염두하여, 도시지역의 택시를 제외한 승용차(경형, 소형, 중형, 대형), 승합차(경형, 소형), RV(소형, 중형)에 적용

    - 1회 주행거리 : 12.35 km (2002년 기준)
    - 미가열 배출 적용 차종 : 승용차(경형, 소형, 중형, 대형) / 승합차(경형, 소형) / RV(소형, 중형)
    - 미가열 배출 적용 지역 : 도시지역(고속도로 구간 제외)

- 산정식

-|Factor $\beta$의 산정식
-|-
$Estimated$ $l_{trip}$|$0.647-0.025 \times l_{trip} - (0.00974 - 0.000385 \times l_{trip}) \times T_a$
$Measured$ $l_{trip}$|$0.698-0.051 \times l_{trip} - (0.01051 - 0.000770 \times l_{trip}) \times T_a$

- $T_a$ : 대기온도
- $Estimated$ $l_{trip}$ : 유럽평균(1985) 약 11.8km / 유럽평균(1998) 약 12.4km(Andre et al., 1998)

In [260]:
l_trip = 12.35 # 국립환경과학원(2007), 도로 이동오염원 대기오염 배출량 산정방법 개선 및 장래 배출량 예측방법 연구
Ta = 12.4 # e-나라지표 (https://www.index.go.kr/unity/potal/main/EachDtlPageDetail.do?idx_cd=1400) 2002년 년 평균 기온

In [261]:
df2['Beta(Estimated)'] = 0.647 - 0.025 * l_trip - (0.00974 - 0.000385 * l_trip) * Ta
# df2['Beta(Measured)'] = 0.698 - 0.051 * l_trip - (0.01051 - 0.000770 * l_trip) * Ta

In [262]:
df2['Beta(Estimated)'].head()

0   0.28
1   0.28
2   0.28
3   0.28
4   0.28
Name: Beta(Estimated), dtype: float64

In [263]:
df2['Beta(Estimated)'].isnull().sum()

0

### $e^{COLD} / e^{HOT}$

In [265]:
df2['fuel'].unique()

array(['휘발유', '경유', 'LPG'], dtype=object)

In [266]:
df2['차량연식'].describe()

count   25117108.00
mean        2014.36
std            6.19
min         1900.00
25%         2011.00
50%         2015.00
75%         2019.00
max         2023.00
Name: 차량연식, dtype: float64

In [270]:
# 1m 31.7s
# 휘발유(자동 제어)
df2.loc[(df2['연료'] == '휘발유') & (df2['차량연식'] < 1991) & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ), 'eCOLD/eHOT_CO'] = 3.7 - 0.09 * Ta
df2.loc[(df2['연료'] == '휘발유') & (df2['차량연식'] >= 1991) & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ), 'eCOLD/eHOT_CO'] = 9.04 - 0.09 * Ta
df2.loc[(df2['연료'] == '휘발유') & (df2['차량연식'] < 1991) & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ), 'eCOLD/eHOT_NOx'] = 1.14 - 0.006 * Ta
df2.loc[(df2['연료'] == '휘발유') & (df2['차량연식'] >= 1991) & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ), 'eCOLD/eHOT_NOx'] = 3.66 - 0.006 * Ta
df2.loc[(df2['연료'] == '휘발유') & (df2['차량연식'] < 1991) & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ), 'eCOLD/eHOT_HC'] = 2.8 - 0.06 * Ta
df2.loc[(df2['연료'] == '휘발유') & (df2['차량연식'] >= 1991) & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ), 'eCOLD/eHOT_HC'] = 12.59 - 0.06 * Ta
df2.loc[df2['연료'] == '휘발유', 'eCOLD/eHOT_PM'] = 1
# 경유
df2.loc[(df2['연료'] == '경유') & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ),'eCOLD/eHOT_CO'] = 1.9 - 0.03 * Ta
df2.loc[(df2['연료'] == '경유') & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ),'eCOLD/eHOT_NOx'] = 1.3 - 0.013 * Ta
df2.loc[(df2['연료'] == '경유') & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ),'eCOLD/eHOT_HC'] = 3.1 - 0.09 * Ta
df2.loc[(df2['연료'] == '경유') & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ),'eCOLD/eHOT_PM'] = 3.1 - 0.1 * Ta
# LPG(액화석유가스)
df2.loc[(df2['연료'] == 'LPG(액화석유가스)') & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ),'eCOLD/eHOT_CO'] = 3.66 - 0.09 * Ta
df2.loc[(df2['연료'] == 'LPG(액화석유가스)') & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ),'eCOLD/eHOT_NOx'] = 0.98 - 0.006 * Ta
df2.loc[(df2['연료'] == 'LPG(액화석유가스)') & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ),'eCOLD/eHOT_HC'] = 2.24 - 0.06 * Ta
df2.loc[(df2['연료'] == 'LPG(액화석유가스)') & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ),'eCOLD/eHOT_PM'] = 1

In [271]:
check_ecoldhot_col = ['eCOLD/eHOT_CO', 'eCOLD/eHOT_NOx', 'eCOLD/eHOT_HC', 'eCOLD/eHOT_PM']
df2[check_ecoldhot_col].isnull().sum()

eCOLD/eHOT_CO     5427867
eCOLD/eHOT_NOx    5427867
eCOLD/eHOT_HC     5427867
eCOLD/eHOT_PM     5394721
dtype: int64

In [272]:
df2[check_ecoldhot_col] = df2[check_ecoldhot_col].fillna(1)
df2[check_ecoldhot_col].isnull().sum()

eCOLD/eHOT_CO     0
eCOLD/eHOT_NOx    0
eCOLD/eHOT_HC     0
eCOLD/eHOT_PM     0
dtype: int64

In [273]:
df2.loc[df2['eCOLD/eHOT_CO'] < 0, 'eCOLD/eHOT_CO'].shape

(0,)

In [274]:
df2.loc[df2['eCOLD/eHOT_NOx'] < 0, 'eCOLD/eHOT_NOx'].shape

(0,)

In [275]:
df2.loc[df2['eCOLD/eHOT_HC'] < 0, 'eCOLD/eHOT_HC'].shape

(0,)

In [276]:
df2.loc[df2['eCOLD/eHOT_PM'] < 0, 'eCOLD/eHOT_PM'].shape

(0,)

### 배출량 계산

In [277]:
df2.columns

Index(['법정동코드', '차종', '차대번호', '차량연식', '배출가스인증번호', '배출가스등급', '차량말소YN', '차종유형',
       '연료', '등급', 'fuel', '차령', '시도', '시구군', '일일평균주행거리', '저감장치부착유무', 'VKT',
       'DF_CO', 'DF_HC', 'DF_PM', 'DF_NOx', 'R_CO', 'R_HC', 'R_PM', 'EFi_CO',
       'EFi_HC', 'EFi_NOx', 'EFi_PM10', 'EFi_PM2_5', 'E_HOT_CO', 'E_HOT_HC',
       'E_HOT_NOx', 'E_HOT_PM10', 'E_HOT_PM2_5', 'Beta(Estimated)',
       'eCOLD/eHOT_CO', 'eCOLD/eHOT_NOx', 'eCOLD/eHOT_HC', 'eCOLD/eHOT_PM'],
      dtype='object')

#### 🔺 계산식 고민
$$E_{COLD :i, j} = \beta_{i, j} \times N_j \times M_j \times e^{HOT}_j \times (e^{COLD} / e^{HOT} \vert_{i, j} - 1)$$
- $M_j$ : 차종 $j$의 주행거리
    - 1년 주행거리로 계산? 
        - VKT
    - 1회 주행거리로 계산?
        - l_trip

In [278]:
#         E      =             B          *     M      *    e(HOT)     * ( e(COLD)/e(HOT)      - 1 )
df2['E_COLD_CO'] = df2['Beta(Estimated)'] * df2['VKT'] * df2['EFi_CO'] * (df2['eCOLD/eHOT_CO'] - 1)
df2['E_COLD_NOx'] = df2['Beta(Estimated)'] * df2['VKT'] * df2['EFi_NOx'] * (df2['eCOLD/eHOT_NOx'] - 1)
df2['E_COLD_HC'] = df2['Beta(Estimated)'] * df2['VKT'] * df2['EFi_HC'] * (df2['eCOLD/eHOT_HC'] - 1)
df2['E_COLD_PM10'] = df2['Beta(Estimated)'] * df2['VKT'] * df2['EFi_PM10'] * (df2['eCOLD/eHOT_PM'] - 1)
df2['E_COLD_PM2_5'] = df2['Beta(Estimated)'] * df2['VKT'] * df2['EFi_PM2_5'] * (df2['eCOLD/eHOT_PM'] - 1)

In [279]:
df2.loc[df2['E_COLD_NOx'] < 0, ['연료', 'Beta(Estimated)', 'VKT', 'EFi_NOx', 'eCOLD/eHOT_NOx', 'E_COLD_NOx']].shape

(1744345, 6)

In [280]:
df2.loc[df2['E_COLD_NOx'] < 0, ['연료', 'Beta(Estimated)', 'VKT', 'EFi_NOx', 'eCOLD/eHOT_NOx', 'E_COLD_NOx']].head()

,연료,Beta(Estimated),VKT,EFi_NOx,eCOLD/eHOT_NOx,E_COLD_NOx
6069,LPG(액화석유가스),0.28,13797.00,0.62,0.91,-223.67
10087,LPG(액화석유가스),0.28,13760.50,0.20,0.91,-73.33
10786,LPG(액화석유가스),0.28,12446.50,0.62,0.91,-201.77
11187,LPG(액화석유가스),0.28,12629.00,0.62,0.91,-204.73
12082,LPG(액화석유가스),0.28,13870.00,0.62,0.91,-224.85


In [281]:
check_E_cold_col = ['E_COLD_CO', 'E_COLD_NOx', 'E_COLD_HC', 'E_COLD_PM10', 'E_COLD_PM2_5']
df2[check_E_HOT_col + check_E_cold_col].head()

,E_HOT_CO,E_HOT_HC,E_HOT_NOx,E_HOT_PM10,E_HOT_PM2_5,E_COLD_CO,E_COLD_NOx,E_COLD_HC,E_COLD_PM10,E_COLD_PM2_5
0,405.90,49.49,29.31,0.01,0.01,88865.91,265.73,7222.95,0.00,0.00
1,30.01,7.37,20.02,0.01,0.01,6569.22,181.56,1076.01,0.00,0.00
2,488.35,59.54,35.26,0.01,0.01,106916.80,319.70,8690.11,0.00,0.00
3,97.81,24.18,108.63,10.70,9.85,0.00,0.00,0.00,0.00,0.00
4,483.28,58.92,34.89,0.01,0.01,105805.98,316.38,8599.82,0.00,0.00


In [282]:
df2[check_E_HOT_col + check_E_cold_col].tail()

,E_HOT_CO,E_HOT_HC,E_HOT_NOx,E_HOT_PM10,E_HOT_PM2_5,E_COLD_CO,E_COLD_NOx,E_COLD_HC,E_COLD_PM10,E_COLD_PM2_5
25117103,0.44,0.01,0.23,0.00,0.00,841.32,162.65,32.50,0.00,0.00
25117104,0.44,0.01,0.23,0.00,0.00,841.32,162.65,32.50,0.00,0.00
25117105,0.41,0.01,0.21,0.00,0.00,791.57,153.03,30.58,0.00,0.00
25117106,13.36,13.36,13.36,13.36,13.36,0.00,0.00,0.00,0.00,0.00
25117107,0.81,0.10,8.40,0.02,0.02,118.28,322.22,27.64,4.18,3.85


In [283]:
df2[check_E_cold_col].isnull().sum()

E_COLD_CO       0
E_COLD_NOx      0
E_COLD_HC       0
E_COLD_PM10     0
E_COLD_PM2_5    0
dtype: int64

## 자동차-휘발유 증발 배출

### 배출량 식
- 3가지
    - 주간증발손실(Diurnal loss)
    - 고온증발손실(Hot and warm soak)
    - 주행손실(Running loss)
- 현재('19 배출량 기준) 휘발유 증발 배출원
    - 주간증발손실, 주행손실 고려
    - 우리나라 휘발유 차량 대부분이 방지설비로 카본 캐니스터를 장착하고 있으므로 $e^{S,HOT}$, $e^{S,WARM}$, 고온증발손실(hot and warm soak) 배출은 없는 것으로 가정

---

$$E_{EVA, VOC:j} = 365 \times N_j \times (e^d + S^c + S^{fi}) + R$$
$S^c = (1-q)(p \times x \times e^{S, HOT} + w \times x \times e^{S,WARM})$

$S^{fi} = q \times e^{fi} \times x$
$$R = m_j(p \times e^{R,HOT} + w \times e^{R,WARM})$$

- $E_{EVA, VOC:j}$ : 차종 j의 증발손실에 의한 VOC 배출량(g/yr)
- $N_j$ : 차종 j의 휘발유 사용 차량 등록대수(대)
- $e^d$ : 금속탱크를 가지고 있는 휘발유 차량의 일중 VOC 배출량(g/day-대)
- $S^c$ : Carburetor 휘발유 차량의 일중 hot and warm soak 배출량(g/day-대)
- $S^{fi}$ : Fuel injection 휘발유 차량의 일중 hot and warm soak 배출량(g/day-대)
- $R$ : Running loss에 의한 휘발유 차량의 연중 배출량(g/yr)
- $q$ : Fuel injection 휘발유 차량의 비율
- $p$ : 가열 엔진 상태로 끝나는 trip의 비율
- $x$ : 일평균 통행횟수 $= V_j / 365 \times l_{trip}$
- $w$ : 미가열 엔진 상태료 끝나는 trip의 비율
- $e^{S,HOT}$ : Hot soak emission의 평균 배출계수
- $e^{S,WARM}$ : Cold and warm soak emission의 평균 배출계수
- $e^{fi}$ : Fuel injection 휘발유 차량의 평균 hot and warm soak 배출계수
- $e^{R,HOT}$ : 휘발유 차량의 평균 hot running loss 배출계수
- $e^{R,WARM}$ : 휘발유 차량의 평균 warm running loss 배출계수
- $m_j$ : 차종 j의 연간 총 주행거리
- $V_j$ : 차종 j의 1대당 평균 연간 주행거리

### 배출계수

#### 🔺 수정 중

#### $e^d$ Diurnal(g/day)
- $e^d$ : 금속탱크를 가지고 있는 휘발유 차량의 일중 VOC 배출량(g/day-대)

##### 🔺 RVP(증기압)
- 참고 COPERT 3(p.80) Table 6.10: Gasoline fuel specifications
    - summer : 60
    - winter : 70
    - 평균 : 65
- 수정 사항(2023.04.20, 최)
    - 국내 과학기술원, 환경부 자료 찾아서 사용 추천

In [284]:
RVP = 65

##### ❗ t_a(기온)
- t_a, t_min, t_rise
    - 참고 COPERT 3(p.72) Table 5.32: Summary of emission factors for estimating evaporative emissions of gasoline vehicles(all RVp in kPa, all temperatures in °C)
        - t_a = (t_max + t_min) / 2
        - t_rise = t_max - t_min
    - 기상자료개방포털(2002년) 연평균 기온
        - https://data.kma.go.kr/stcs/grnd/grndTaList.do?pgmNo=70
        - 7.5°C(최저), 12.4°C(평균), 17.9°C(최고)
    - 기상자료개방포털(2022년) 연평균 기온
        - https://data.kma.go.kr/stcs/grnd/grndTaList.do?pgmNo=70
        - 8°C(최저), 12.9°C(평균), 18.6°C(최고)

In [285]:
t_min = 8 # 평균 최저 온도
t_max = 18.6
t_a = (t_min + t_max) / 2
t_rise = 10.6 # 평균 상승 온도

In [286]:
df2.loc[df2['차량연식'] < 1991, 'e_d'] = 9.1 * np.exp( 0.0158 * (RVP - 61.2) + 0.0574 * (t_min - 22.5) + 0.0614 * (t_rise - 11.7) )
df2.loc[df2['차량연식'] >= 1991, 'e_d'] = 0.2 * (9.1 * np.exp( 0.0158 * (RVP - 61.2) + 0.0574 * (t_min - 22.5) + 0.0614 * (t_rise - 11.7) ))

In [287]:
df2['e_d'].isnull().sum()

0

#### $e^{R,HOT}$ Hot running loss(g/km)
- $e^{R,HOT}$ : 휘발유 차량의 평균 hot running loss 배출계수

In [288]:
df2.loc[df2['차량연식'] < 1991, 'e_RHOT'] = 0.136 * np.exp( - 5.967 + 0.04259 * RVP + 0.1773 * t_a)
df2.loc[df2['차량연식'] >= 1991, 'e_RHOT'] = 0.1 * (0.136 * np.exp( - 5.967 + 0.04259 * RVP + 0.1773 * t_a))

In [289]:
df2['e_RHOT'].isnull().sum()

0

#### $e^{S,HOT}$ Hot soak(g/procedure)
- $e^{S,HOT}$ : Hot soak emission의 평균 배출계수

In [290]:
df2.loc[df2['차량연식'] < 1991, 'e_SHOT'] = 3.0042 * np.exp(0.02 * RVP)
df2.loc[df2['차량연식'] >= 1991, 'e_SHOT'] = 0.3 * np.exp(-2.14 + 0.02302 * RVP + 0.09408 * t_a)

#### $e^{S,WARM}$ Waram soak(g/procedure)
- $e^{S,WARM}$ : Cold and warm soak emission의 평균 배출계수

In [291]:
df2.loc[df2['차량연식'] < 1991, 'e_SWARM'] = np.exp(-1.644 + 0.01993 * RVP + 0.07521 * t_a)
df2.loc[df2['차량연식'] >= 1991, 'e_SWARM'] = 0.2 * np.exp(-2.41 + 0.02302 * RVP + 0.09408 * t_a)

#### $e^{fi}$ Warm and hot soak for Fuel injected vehicle(g/procedure)
- $e^{fi}$ : Fuel injection 휘발유 차량의 평균 hot and warm soak 배출계수

In [292]:
df2.loc[df2['차량연식'] < 1991, 'e_fi'] = 0.7
df2.loc[df2['차량연식'] >= 1991, 'e_fi'] = 0

#### $e^{R,WARM}$ Warm running loss(g/km)
- $e^{R,HOT}$ : 휘발유 차량의 평균 hot running loss 배출계수

In [293]:
df2.loc[df2['차량연식'] < 1991, 'e_RWARM'] = 0.1 * np.exp(-5.967 + 0.04259 * RVP + 0.1773 * t_a)
df2.loc[df2['차량연식'] >= 1991, 'e_RWARM'] = 0.1 * (0.1 * np.exp(-5.967 + 0.04259 * RVP + 0.1773 * t_a))

In [294]:
df2['e_RWARM'].isnull().sum()

0

### $R$
$R = m_j(p \times e^{R,HOT} + w \times e^{R,WARM})$
- Running loss에 의한 휘발유 차량의 연중 배출량(g/yr)
- $m_j$ : 차종 j의 연간 총 주행거리
- $p$ : 가열 엔진 상태로 끝나는 trip의 비율
- $e^{R,HOT}$ : 휘발유 차량의 평균 hot running loss 배출계수
- $w$ : 미가열 엔진 상태로 끝나는 trip의 비율
- $e^{R,WARM}$ : 휘발유 차량의 평균 warm running loss 배출계수

#### ❓ 확인 중
- 수정 사항(2023.04.20, 최)
    - 찾는 중

In [295]:
p = 0.5 # 가열 엔진 상태로 끝나는 trip의 비율
w = 0.5 # 미가열 엔진 상태로 끝나는 trip의 비율

In [296]:
df2['R'] = df2['VKT'] * (p * df2['e_RHOT'] + w * df2['e_RWARM'])

In [297]:
df2['R'].isnull().sum()

0

### $S^{fi}$
$S^{fi} = q \times e^{fi} \times x$
- Fuel injection 휘발유 차량의 일중 hot and warm soak 배출량(g/day-대)
- $q$ : Fuel injection 휘발유 차량의 비율
- $e^{fi}$ : Fuel injection 휘발유 차량의 평균 hot and warm soak 배출계수
- $x$ : 일평균 통행횟수 $= V_j / 365 \times l_{trip}$
- $V_j$ : 차종 j의 1대당 평균 연간 주행거리

In [298]:
# 8.9s
not_injection = df2[(df2['fuel'] == '휘발유') & (df2['차량연식'] < 1991)].shape[0]
injection = df2[(df2['fuel'] == '휘발유') & (df2['차량연식'] >= 1991)].shape[0]
# Fuel injection 휘발유 차량의 비율
q = injection / (injection + not_injection)
q

0.9931614868162287

In [299]:
# x = df2['VKT'] / (365 * l_trip)
df2['S_fi'] = q * df2['e_fi'] * ( df2['VKT'] / (365 * l_trip) )

### $S^c$
$S^c = (1-q)(p \times x \times e^{S, HOT} + w \times x \times e^{S,WARM})$
- Carburetor 휘발유 차량의 일중 hot and warm soak 배출량(g/day-대)
- $q$ : Fuel injection 휘발유 차량의 비율
- $p$ : 가열 엔진 상태로 끝나는 trip의 비율
- $x$ : 일평균 통행횟수 $= V_j / 365 \times l_{trip}$
- $e^{S,HOT}$ : Hot soak emission의 평균 배출계수
- $w$ : 미가열 엔진 상태료 끝나는 trip의 비율
- $x$ : 일평균 통행횟수 $= V_j / 365 \times l_{trip}$
- $V_j$ : 차종 j의 1대당 평균 연간 주행거리
- $e^{S,WARM}$ : Cold and warm soak emission의 평균 배출계수

In [300]:
# x = df2['VKT'] / (365 * l_trip)
df2['S_c'] = (1-q) * (p * (df2['VKT'] / (365 * l_trip)) * df2['e_SHOT'] + w * (df2['VKT'] / (365 * l_trip)) * df2['e_SWARM'])

### $E_{EVA,VOC}$
$E_{EVA, VOC:j} = 365 \times N_j \times (e^d + S^c + S^{fi}) + R$
- $N_j$ : 차종 j의 휘발유 사용 차량 등록대수(대)
- $e^d$ : 금속탱크를 가지고 있는 휘발유 차량의 일중 VOC 배출량(g/day-대)
- $S^c$ : Carburetor 휘발유 차량의 일중 hot and warm soak 배출량(g/day-대)
- $S^{fi}$ : Fuel injection 휘발유 차량의 일중 hot and warm soak 배출량(g/day-대)
- $R$ : Running loss에 의한 휘발유 차량의 연중 배출량(g/yr)

In [301]:
df2['E_EVA_VOC'] = 365 * df2['e_d'] + df2['S_c'] + df2['S_fi'] + df2['R']

In [302]:
df2['E_EVA_VOC'].isnull().sum()

0

## 배출량 합계
$E_{total}(kg) = E_{HOT}(kg) + E_{COLD}(g) + E_{EVAP}(g)$

In [303]:
df2[df2['E_COLD_NOx'] < 0].shape

(1744345, 54)

In [304]:
# E_COLD_NOx 음수 -> 0으로 처리(2023.04.24 from 최이사님)
df2.loc[df2['E_COLD_NOx'] < 0, 'E_COLD_NOx'] = 0

In [305]:
df2[df2['E_COLD_NOx'] < 0].shape

(0, 54)

In [306]:
check_E_HOT_col, check_E_cold_col

(['E_HOT_CO', 'E_HOT_HC', 'E_HOT_NOx', 'E_HOT_PM10', 'E_HOT_PM2_5'],
 ['E_COLD_CO', 'E_COLD_NOx', 'E_COLD_HC', 'E_COLD_PM10', 'E_COLD_PM2_5'])

In [307]:
df2.loc[(df2['fuel'] == '휘발유') & (df2['E_COLD_PM10'] != 0)].shape

(0, 54)

In [308]:
df2['fuel'].unique()

array(['휘발유', '경유', 'LPG'], dtype=object)

In [309]:
# 26.5s
fuel = '휘발유'
df2.loc[df2['fuel'] == fuel, 'E_CO_total'] = df2.loc[df2['fuel'] == fuel , 'E_HOT_CO'] + df2.loc[df2['fuel'] == fuel , 'E_COLD_CO']/1000
df2.loc[df2['fuel'] == fuel, 'E_HC_total'] = df2.loc[df2['fuel'] == fuel , 'E_HOT_HC'] + df2.loc[df2['fuel'] == fuel , 'E_COLD_HC']/1000 + df2.loc[df2['fuel'] == fuel , 'E_EVA_VOC']/1000
df2.loc[df2['fuel'] == fuel, 'E_NOx_total'] = df2.loc[df2['fuel'] == fuel , 'E_HOT_NOx'] + df2.loc[df2['fuel'] == fuel , 'E_COLD_NOx']/1000
df2.loc[df2['fuel'] == fuel, 'E_PM10_total'] = df2.loc[df2['fuel'] == fuel , 'E_HOT_PM10'] + df2.loc[df2['fuel'] == fuel , 'E_COLD_PM10']/1000
df2.loc[df2['fuel'] == fuel, 'E_PM2_5_total'] = df2.loc[df2['fuel'] == fuel , 'E_HOT_PM2_5'] + df2.loc[df2['fuel'] == fuel , 'E_COLD_PM2_5']/1000

In [310]:
# 22.7s
fuel = '경유'
df2.loc[df2['fuel'] == fuel, 'E_CO_total'] = df2.loc[df2['fuel'] == fuel , 'E_HOT_CO'] + df2.loc[df2['fuel'] == fuel , 'E_COLD_CO']/1000
df2.loc[df2['fuel'] == fuel, 'E_HC_total'] = df2.loc[df2['fuel'] == fuel , 'E_HOT_HC'] + df2.loc[df2['fuel'] == fuel , 'E_COLD_HC']/1000
df2.loc[df2['fuel'] == fuel, 'E_NOx_total'] = df2.loc[df2['fuel'] == fuel , 'E_HOT_NOx'] + df2.loc[df2['fuel'] == fuel , 'E_COLD_NOx']/1000
df2.loc[df2['fuel'] == fuel, 'E_PM10_total'] = df2.loc[df2['fuel'] == fuel , 'E_HOT_PM10'] + df2.loc[df2['fuel'] == fuel , 'E_COLD_PM10']/1000
df2.loc[df2['fuel'] == fuel, 'E_PM2_5_total'] = df2.loc[df2['fuel'] == fuel , 'E_HOT_PM2_5'] + df2.loc[df2['fuel'] == fuel , 'E_COLD_PM2_5']/1000

In [311]:
# 20.0s
fuel = 'LPG'
df2.loc[df2['fuel'] == fuel, 'E_CO_total'] = df2.loc[df2['fuel'] == fuel , 'E_HOT_CO'] + df2.loc[df2['fuel'] == fuel , 'E_COLD_CO']/1000
df2.loc[df2['fuel'] == fuel, 'E_HC_total'] = df2.loc[df2['fuel'] == fuel , 'E_HOT_HC'] + df2.loc[df2['fuel'] == fuel , 'E_COLD_HC']/1000
df2.loc[df2['fuel'] == fuel, 'E_NOx_total'] = df2.loc[df2['fuel'] == fuel , 'E_HOT_NOx'] + df2.loc[df2['fuel'] == fuel , 'E_COLD_NOx']/1000
df2.loc[df2['fuel'] == fuel, 'E_PM10_total'] = df2.loc[df2['fuel'] == fuel , 'E_HOT_PM10'] + df2.loc[df2['fuel'] == fuel , 'E_COLD_PM10']/1000
df2.loc[df2['fuel'] == fuel, 'E_PM2_5_total'] = df2.loc[df2['fuel'] == fuel , 'E_HOT_PM2_5'] + df2.loc[df2['fuel'] == fuel , 'E_COLD_PM2_5']/1000

In [312]:
check_E_col = ['E_CO_total', 'E_HC_total', 'E_NOx_total', 'E_PM10_total', 'E_PM2_5_total']
df2[check_E_col].head()

,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total
0,494.77,58.74,29.57,0.01,0.01
1,36.57,10.58,20.21,0.01,0.01
2,595.27,70.38,35.58,0.01,0.01
3,97.81,24.18,108.63,10.70,9.85
4,589.08,69.66,35.21,0.01,0.01


In [314]:
df2.loc[df2['fuel'] == '휘발유', check_E_col + ['E_EVA_VOC']].head()

,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total,E_EVA_VOC
0,494.77,58.74,29.57,0.01,0.01,2030.80
1,36.57,10.58,20.21,0.01,0.01,2129.61
2,595.27,70.38,35.58,0.01,0.01,2151.98
4,589.08,69.66,35.21,0.01,0.01,2144.52
6,494.77,58.74,29.57,0.01,0.01,2030.80


In [315]:
df2.loc[df2['fuel'] == '경유', check_E_col].head()

,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total
3,97.81,24.18,108.63,10.70,9.85
5,114.72,28.36,127.41,12.55,11.55
9,74.78,18.49,83.06,8.18,7.53
11,105.14,25.99,116.78,11.50,10.58
16,122.66,40.40,573.17,37.57,34.56


In [316]:
df2.loc[df2['fuel'] == 'LPG', check_E_col].head()

,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total
6069,68.95,6.50,8.57,0.00,0.00
10087,15.77,0.67,2.81,0.00,0.00
10786,62.20,5.86,7.73,0.00,0.00
11187,63.11,5.95,7.85,0.00,0.00
12082,69.32,6.53,8.62,0.00,0.00


In [317]:
df2.loc[(df2['fuel'] == '경유') & (df2['차종'] == '승용'), DF_col + check_E_col].head()

,fuel,차종,차종유형,차량연식,차령,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total
29,경유,승용,중형,1972,51,2.55,0.23,19.30,1.03,0.94
31,경유,승용,중형,1973,50,2.59,0.24,19.54,1.04,0.96
43,경유,승용,중형,1975,48,2.69,0.25,20.32,1.08,0.99
60,경유,승용,소형,1983,40,2.83,0.26,21.41,1.14,1.05
73,경유,승용,중형,1975,48,2.72,0.25,20.57,1.09,1.01


In [318]:
df2.loc[(df2['fuel'] == '경유') & (df2['차종'] == '화물'), DF_col + check_E_col].head()

,fuel,차종,차종유형,차량연식,차령,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total
3,경유,화물,중형,1971,52,97.81,24.18,108.63,10.70,9.85
5,경유,화물,중형,1976,47,114.72,28.36,127.41,12.55,11.55
9,경유,화물,중형,1967,56,74.78,18.49,83.06,8.18,7.53
11,경유,화물,중형,1976,47,105.14,25.99,116.78,11.50,10.58
16,경유,화물,대형,1970,53,122.66,40.40,573.17,37.57,34.56


In [319]:
df2.loc[(df2['fuel'] == '휘발유') & (df2['차종'] == '승용'), DF_col + check_E_col].head()

,fuel,차종,차종유형,차량연식,차령,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total
0,휘발유,승용,소형,1976,47,494.77,58.74,29.57,0.01,0.01
1,휘발유,승용,경형,1968,55,36.57,10.58,20.21,0.01,0.01
2,휘발유,승용,소형,1976,47,595.27,70.38,35.58,0.01,0.01
4,휘발유,승용,소형,1986,37,589.08,69.66,35.21,0.01,0.01
6,휘발유,승용,소형,1976,47,494.77,58.74,29.57,0.01,0.01


In [320]:
df2.loc[(df2['fuel'] == '휘발유') & (df2['차종'] == '화물'), DF_col + check_E_col].head()

,fuel,차종,차종유형,차량연식,차령,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total
61,휘발유,화물,소형,1973,50,66.73,11.69,15.78,0.01,0.01
87,휘발유,화물,대형,1980,43,90.74,24.49,125.98,9.93,9.13
96,휘발유,화물,소형,1976,47,57.79,10.32,13.67,0.01,0.01
141,휘발유,화물,중형,1972,51,186.92,48.93,259.51,20.45,18.82
159,휘발유,화물,대형,1962,61,90.74,24.49,125.98,9.93,9.13


### issue
- LPG : E_COLD_NOx 값 음수 -> 0으로 처리

In [321]:
df2.loc[df2['fuel'] == 'LPG', DF_col + check_E_HOT_col + check_E_cold_col + check_E_col].head()

,fuel,차종,차종유형,차량연식,차령,E_HOT_CO,E_HOT_HC,E_HOT_NOx,E_HOT_PM10,E_HOT_PM2_5,E_COLD_CO,E_COLD_NOx,E_COLD_HC,E_COLD_PM10,E_COLD_PM2_5,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total
6069,LPG,승용,소형,1983,40,48.33,5.72,8.57,0.00,0.00,20625.83,0.00,783.64,0.00,0.00,68.95,6.50,8.57,0.00,0.00
10087,LPG,승용,소형,2002,21,11.05,0.59,2.81,0.00,0.00,4717.69,0.00,81.29,0.00,0.00,15.77,0.67,2.81,0.00,0.00
10786,LPG,승용,소형,1982,41,43.60,5.16,7.73,0.00,0.00,18606.90,0.00,706.93,0.00,0.00,62.20,5.86,7.73,0.00,0.00
11187,LPG,승용,소형,1982,41,44.23,5.23,7.85,0.00,0.00,18879.73,0.00,717.30,0.00,0.00,63.11,5.95,7.85,0.00,0.00
12082,LPG,승용,소형,1982,41,48.58,5.75,8.62,0.00,0.00,20734.96,0.00,787.78,0.00,0.00,69.32,6.53,8.62,0.00,0.00


In [322]:
df2.loc[df2['fuel'] == 'LPG', DF_col + check_E_cold_col + check_E_col].head()

,fuel,차종,차종유형,차량연식,차령,E_COLD_CO,E_COLD_NOx,E_COLD_HC,E_COLD_PM10,E_COLD_PM2_5,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total
6069,LPG,승용,소형,1983,40,20625.83,0.00,783.64,0.00,0.00,68.95,6.50,8.57,0.00,0.00
10087,LPG,승용,소형,2002,21,4717.69,0.00,81.29,0.00,0.00,15.77,0.67,2.81,0.00,0.00
10786,LPG,승용,소형,1982,41,18606.90,0.00,706.93,0.00,0.00,62.20,5.86,7.73,0.00,0.00
11187,LPG,승용,소형,1982,41,18879.73,0.00,717.30,0.00,0.00,63.11,5.95,7.85,0.00,0.00
12082,LPG,승용,소형,1982,41,20734.96,0.00,787.78,0.00,0.00,69.32,6.53,8.62,0.00,0.00


In [323]:
df2.columns

Index(['법정동코드', '차종', '차대번호', '차량연식', '배출가스인증번호', '배출가스등급', '차량말소YN', '차종유형',
       '연료', '등급', 'fuel', '차령', '시도', '시구군', '일일평균주행거리', '저감장치부착유무', 'VKT',
       'DF_CO', 'DF_HC', 'DF_PM', 'DF_NOx', 'R_CO', 'R_HC', 'R_PM', 'EFi_CO',
       'EFi_HC', 'EFi_NOx', 'EFi_PM10', 'EFi_PM2_5', 'E_HOT_CO', 'E_HOT_HC',
       'E_HOT_NOx', 'E_HOT_PM10', 'E_HOT_PM2_5', 'Beta(Estimated)',
       'eCOLD/eHOT_CO', 'eCOLD/eHOT_NOx', 'eCOLD/eHOT_HC', 'eCOLD/eHOT_PM',
       'E_COLD_CO', 'E_COLD_NOx', 'E_COLD_HC', 'E_COLD_PM10', 'E_COLD_PM2_5',
       'e_d', 'e_RHOT', 'e_SHOT', 'e_SWARM', 'e_fi', 'e_RWARM', 'R', 'S_fi',
       'S_c', 'E_EVA_VOC', 'E_CO_total', 'E_HC_total', 'E_NOx_total',
       'E_PM10_total', 'E_PM2_5_total'],
      dtype='object')

In [324]:
df2['E_COLD_CO'].dtype

dtype('float64')

In [325]:
df2.loc[df2['E_CO_total'] < 0].shape

(0, 59)

In [326]:
df2.loc[df2['E_HC_total'] < 0].shape

(0, 59)

In [327]:
df2.loc[df2['E_PM10_total'] < 0].shape

(0, 59)

In [328]:
df2.loc[df2['E_PM2_5_total'] < 0].shape

(0, 59)

In [329]:
df2.loc[df2['E_NOx_total'] < 0, ['E_NOx_total', 'E_COLD_NOx']].shape

(0, 2)

## 황산화물 배출량

### [출력] 배출량
- 연료 : 휘발유, 경유, LPG(액화석유가스)
- 물질 : CO, HC, NOx, PM10, PM2.5

In [330]:
df2.columns

Index(['법정동코드', '차종', '차대번호', '차량연식', '배출가스인증번호', '배출가스등급', '차량말소YN', '차종유형',
       '연료', '등급', 'fuel', '차령', '시도', '시구군', '일일평균주행거리', '저감장치부착유무', 'VKT',
       'DF_CO', 'DF_HC', 'DF_PM', 'DF_NOx', 'R_CO', 'R_HC', 'R_PM', 'EFi_CO',
       'EFi_HC', 'EFi_NOx', 'EFi_PM10', 'EFi_PM2_5', 'E_HOT_CO', 'E_HOT_HC',
       'E_HOT_NOx', 'E_HOT_PM10', 'E_HOT_PM2_5', 'Beta(Estimated)',
       'eCOLD/eHOT_CO', 'eCOLD/eHOT_NOx', 'eCOLD/eHOT_HC', 'eCOLD/eHOT_PM',
       'E_COLD_CO', 'E_COLD_NOx', 'E_COLD_HC', 'E_COLD_PM10', 'E_COLD_PM2_5',
       'e_d', 'e_RHOT', 'e_SHOT', 'e_SWARM', 'e_fi', 'e_RWARM', 'R', 'S_fi',
       'S_c', 'E_EVA_VOC', 'E_CO_total', 'E_HC_total', 'E_NOx_total',
       'E_PM10_total', 'E_PM2_5_total'],
      dtype='object')

# 통계

## 연료/시도 별 배출량 합계

In [331]:
groupby1 = df2.groupby(['시도']).agg({'E_CO_total':'sum', 'E_HC_total':'sum', 'E_NOx_total':'sum', 'E_PM10_total':'sum', 'E_PM2_5_total':'sum'})
groupby1

,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total
시도,,,,,
강원도,5065170.79,1106593.67,7844359.27,622818.81,610874.79
경기도,39499252.41,9398742.91,58924335.41,5532122.26,5460001.11
경상남도,11983104.09,2702887.56,18752528.13,1549158.38,1521916.78
경상북도,10333279.06,2155106.82,17126530.89,1158165.43,1128684.40
광주광역시,4081039.33,979307.25,6577765.71,578815.94,569461.39
대구광역시,7487903.76,1792186.66,10433355.28,1065562.63,1051862.77
대전광역시,4301996.11,955123.79,5953671.84,537486.76,529736.27
부산광역시,9602758.11,2417191.20,14485308.82,1455406.06,1435577.60
서울특별시,22368860.20,5092821.84,24304184.51,2729339.42,2695905.30


In [332]:
today_date = datetime.today().strftime("%Y.%m.%d_%p.%I.%M.%S")
today_date

'2023.06.09_AM.10.58.42'

In [200]:
# groupby1.to_excel(os.path.join(analysis_fold, f'BD1-3_통계_result_4등급_시도별_CAPSS매연량합계({today_date}).xlsx'), sheet_name='시도_시군구')

# [출력] 경유, 휘발유, LPG 차량 배출량

In [ ]:
# 컬럼명 정리

In [ ]:
# # about
# df2.to_csv(os.path.join(df_fold, '경유_휘발유_LPG차량_배출량.csv'), index=False)

# 코드 마지막